In [101]:
from wallet import Wallet
from accounts import SHMAccount, FHMAccount, Account
from ecc import PrivateKey, S256Point
from transactions import MultiSigTransaction
from bip32 import Xtended_pubkey, Xtended_privkey
from constants import *

### Wallets


- CorporateSuperWallet is the main wallet. I AM THINKING TO MAKE MAIN WALLET A COMPANY WALLET INSTEAD, WHERE THERE IS NO PRIVATE KEYS.
- CorporateWallet: cosigners at the top corporate level.
- StoreWallet: responsible for only address generation for payments in each store.
- DailySafeWallet: Multisignature single-herarchical wallet. Manageble (see balance and create transactions) by everybody but AssistantManagerWallet who only can authorize in emergency events.
- WeeklySafeWallet: same as DailySafeWallet with the exception that no AssistantManagerWallet is part of this wallet.
- CorporateSafeWallet: Multisignature fully-herarchical wallet. Supported on the non_hardened keys within tge year account which is a hardened derived account.
- ManagerWallet: takes part in the DailySafeWallet and the WeeklySafeWallet.
- AssistantManagerWallet: only a cosigner wallet. This is the most basic and simple wallet type.


In [102]:
class CorporateSuperWallet(Wallet):
    
    def get_daily_safe_pubkey(self):
        path = "m/44H/0H/1H"
        daily_safes_acc = self.get_child_from_path(path)
        return daily_safes_acc.private_key_obj.point.sec()
    
    
    def create_store_account(self, store_code):
        """
        Returns the master private key of the store with index 'store_code'.
        store_code: Integer. The code designated to the store. It is recomended that 
        this code matches that one already stablished by the company.
        """
        path = "m/44H/0H/0H/"+str(store_code)
        print(path)
        store_account = self.get_child_from_path(path)
        return store_account
    
    def request_pubkeys_daily_safe(self,store_code, m=2,n=6):
        """
        Returns the message to start requesting the public keys necessary to setup a
        daily safe.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction.
        store_code: Integer. The code designated to the store. It is recomended that 
        this code matches that already stablished by the company.
        """
        #save in the database the unique_id, public keys, m, n, and the store_code.
        return 
    
    def create_daily_safe_wallet(self, public_key_list, m=2, n=6,addr_type="p2wsh",
                                 testnet=False, segwit=True):
        """
        This is a watch-only wallet by definition. CHANGE THIS TO BE ABLE TO SIGN TX!
        Returns a Single-Herarchical multisignature master wallet based on the public keys 
        and the master_pubkey.
        public_key_list: the list of public keys of the keys that can sign the transaction.
        the length of the list must be 6 according to the protocol. However, a different 
        length can be stablished.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction. Must match with 
        public_key_list's length-1.
        """
        
        path = "m/44H/0H/1H"
        daily_safes_acc = self.get_child_from_path(path)
        print(daily_safes_acc)
        shm_account = SHMAccount(m,n, str(daily_safes_acc.xtended_public_key), public_key_list, _privkey=None,
                                 addr_type=addr_type,testnet=testnet, segwit=segwit)
        return shm_account
    
    def create_weekly_safe_wallet(self,master_pubkey, public_key_list, m=2, n=5,addr_type="p2wsh",
                                 testnet=False, segwit=True):
        """
        This is a watch-only wallet by definition. CHANGE THIS TO BE ABLE TO SIGN TX!
        Returns a Single-Herarchical multisignature master wallet based on the public keys 
        and the master_pubkey.
        public_key_list: the list of public keys of the keys that can sign the transaction.
        the length of the list must be 6 according to the protocol. However, a different 
        length can be stablished.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction. Must match with 
        public_key_list's length-1.
        """
        path = "m/44H/0H/2H"
        weekly_safes_acc = self.get_child_from_path(path)
        shm_account = SHMAccount(m,n, str(weekly_safes_acc.xtended_public_key), public_key_list, _privkey=None,
                                 addr_type=addr_type,testnet=testnet, segwit=segwit)
        return shm_account
    
    def create_corporate_safe(self, index):
        return 
    
    def get_store_deposit_account(self, store_index):
        return
    
    def get_year_xpub(self):
        path = "m/44H/0H/3H"
        year_account = self.get_child_from_path(path)
        return year_account.xtended_public_key
    
    def accept_corporate_safe_invite(self,invite, name):
        """
        accepts the invitation from a SuperCorporateWallet to participate in a daily-safe wallet.
        The acceptance must be through a private key that matches one of the public keys that are
        part of the SHM wallet passed in the invite.
        invite: SHMAccount object. The wallet that we are going to be part of.
        """
        path = "m/44H/0H/3H"
        year_safe_acc = self.get_child_from_path(path)
        print(f"accept_safe_wallet_invite: priv: {year_safe_acc.xtended_key}; pubkey: {{year_safe_acc.xtended_public_key}}")

        master_pubkey_list = invite["master_pubkey_list"]
        m = invite["m"]
        n = invite["n"]
        addr_type = invite["addr_type"]
        segwit = invite["segwit"]
        testnet = invite["testnet"]
   
            
        safe_wallet = HDMWallet( name, master_pubkey_list, year_safe_acc,m, n, addr_type, testnet,segwit)
        
        return safe_wallet

In [103]:
class StoreWallet(Wallet):
    
    def __init__(self, xpriv):
        if not isinstance(xpriv, Xtended_privkey): xpriv = Xtended_privkey.parse( xpriv )
            
        super().__init__(depth=xpriv.depth, fingerprint=xpriv.fingerprint, index=xpriv.index, 
                         chain_code=xpriv.chain_code,private_key=xpriv.private_key, testnet=xpriv.testnet)
        
    
    def create_receiving_address(self,index, addr_type = "p2pkh"):
        if addr_type.lower()   ==   "p2pkh":     _type  = P2PKH
        elif addr_type.lower() ==  "p2wpkh":     _type  = P2WPKH
        elif addr_type.lower() ==    "p2sh":     _type  = P2SH
        elif addr_type.lower() ==   "p2wsh":     _type  = P2WSH
        elif addr_type.lower() == "p2sh_p2wpkh": _type  = P2SH_P2WPKH
        elif addr_type.lower() == "p2sh_p2wsh":  _type  = P2SH_P2WSH
        else: raise Exception(f"{addr_type} is NOT a valid type of address.")
        receiving_path = "m/"
        #i = self.get_i(receiving_path, index)
        i = index
        path = receiving_path + str(i)
        print(f"Deposit address's Path: {path}")
        receiving_xtended_acc = self.get_child_from_path(path)
        account = Account(int.from_bytes(receiving_xtended_acc.private_key,"big"),addr_type, self.testnet )
        self.start_conn()
        self.db.new_address(account.address,receiving_path,i,FALSE, _type, self.get_xtended_key())
        self.close_conn()
        return account

In [104]:
words = "engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning"
corporate1 = CorporateSuperWallet.recover_from_words(words, 256, "RobertPauslon",True)

store_504_account = corporate1.create_store_account(504)

store_504_wallet = StoreWallet(store_504_account)
store_504_wallet

8418572761394767459444418608530371567217744336876557229650655736270882421023962287566834689847418048023007033142570964655978481626857402470209587761075589
m/44H/0H/0H/504


tprv8huKEhbYUbj1AKLqzjCww2aS7FCYhUJ5FKkarkh3jqFj7nx2CuKwxCeD9Yc6YkSpddgfh15WifYaCDWUfELninzHYiM9ayz7T1SqXQ4zqhp

In [41]:
address = store_504_wallet.create_receiving_address(1)

Deposit address's Path: m/1
connection with database made.
INSERT OR IGNORE INTO Addresses (address, path, acc_index, change_addr, created, type, wallet, safe_index)
 VALUES("msEqY81jTYjjcw9JKDqiehn8sBWN4U2xBp", "m/", 1, 0, 1598637590, 2, "tprv8huKEhbYUbj1AKLqzjCww2aS7FCYhUJ5FKkarkh3jqFj7nx2CuKwxCeD9Yc6YkSpddgfh15WifYaCDWUfELninzHYiM9ayz7T1SqXQ4zqhp", 0) ;


In [32]:
address.address

'msEqY81jTYjjcw9JKDqiehn8sBWN4U2xBp'

In [105]:
class ManagerWallet(Wallet):
    
    def get_daily_safe_pubkey(self):
        path = "m/44H/0H/1H"
        daily_safes_acc = self.get_child_from_path(path)
        return daily_safes_acc.private_key_obj.point.sec()
    
    def accept_invite(self,invite, name):
        """
        accepts the invitation from a SuperCorporateWallet to participate in a daily-safe wallet.
        The acceptance must be through a private key that matches one of the public keys that are
        part of the SHM wallet passed in the invite.
        invite: SHMAccount object. The wallet that we are going to be part of.
        """
        path = "m/44H/0H/1H"
        daily_safes_acc = self.get_child_from_path(path)
        print(daily_safes_acc.private_key)
        
        public_key_list = invite["public_key_list"]
        master_pubkey = invite["master_pubkey"]
        m = invite["m"]
        n = invite["n"]
        addr_type = invite["addr_type"]
        segwit = invite["segwit"]
        testnet = invite["testnet"]
        
        safe_wallet = SHDSafeWallet.from_privkey_masterpubkey( name, public_key_list, master_pubkey, 
                                                              daily_safes_acc.private_key,m, n, addr_type, testnet,
                                                              segwit)
     
        return safe_wallet
        

In [5]:
words = "engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning"
manager1 = ManagerWallet.recover_from_words(words, 256, "RobertPauslon",True)

words = "client sudden sunset borrow pupil rely sand girl prefer movie bachelor guilt giraffe glove much strong dizzy switch ill silent goddess crumble goat power"
manager2 = ManagerWallet.recover_from_words(words,entropy=256,passphrase="RobertPaulson",testnet=True)


public_keys = [ manager1.get_daily_safe_pubkey(), manager2.get_daily_safe_pubkey(), PrivateKey(int.from_bytes(b"Ashley Serna","little")).point.sec()]


8418572761394767459444418608530371567217744336876557229650655736270882421023962287566834689847418048023007033142570964655978481626857402470209587761075589
6018716354834777059445516500463325153454936853151215624488541646839939367247233986759821493097378851875115552924255831805062990606698886137563075847103799


In [41]:
corp_wallet=CorporateSuperWallet.generate_random(testnet=True)

Copy these words for future recovery:
b'raccoon unhappy erode sign meadow cave stadium smooth excess panda yellow key'
3967057481929135490959568468773937162564863802327639838595367726563211014079398303801697444580104771319233996176199962751504258379722189521574850955374009


In [42]:
invite = corp_wallet.create_daily_safe_wallet(public_keys,n=4,testnet=True)

tprv8gb1zANReLHhcwh7erfL1cCLtBrp1xwYcwoFDNHmS5pqipGa25AWAKYHFvAZijZz19JW2yxnzb2w2kmtBzp4STPE6dqHo5Zk9UZ9pnLTWdz


In [43]:
daily_wallet_manager1 = manager1.accept_daily_safe_wallet(invite)

AttributeError: 'ManagerWallet' object has no attribute 'accept_daily_safe_wallet'

In [ ]:
daily_wallet_manager2 = manager2.accept_daily_safe_wallet(invite)

In [ ]:
deposit_m1 = daily_wallet_manager1.get_deposit_account(200820)
deposit_m1.address

In [ ]:
deposit_m2 = daily_wallet_manager2.get_deposit_account(200820)
deposit_m2.address

In [ ]:
prev_tx_id_list = ["d629b979a1d2b2eb797c0f37adee6920fac4a77b4232baaa975c4b499e862f1c"]
my_tx = MultiSigTransaction.create(prev_tx_id_list, [("mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8",777)], deposit_m1, fee=None, 
                 segwit=True)

In [ ]:
my_tx_signed_by2 = MultiSigTransaction.sign_received_tx(my_tx,deposit_m2)

In [ ]:
my_tx_signed_by2.serialize().hex()

In [106]:
from constants import *
from wallet import Wallet
from accounts import SHMAccount, MultSigAccount
from ecc import PrivateKey, S256Point
from transactions import MultiSigTransaction
import datetime
from bip32 import Xtended_pubkey, Xtended_privkey
from blockcypher import get_address_details, get_transaction_details
from blockcypher import pushtx
import threading
from wallet_database_sqlite3 import Sqlite3Wallet, Sqlite3Environment


class SHDSafeWallet(Wallet):
    """
    SafeWallet is a Multisignature SHD wallet.
    SHDM Single Herarchical Deterministic Multi-Signature Wallet
    """
    def __init__(self, name, public_key_list, m=2, n=6,addr_type="p2wsh",_privkey=None, master_pubkey=None, 
                 master_privkey=None, testnet=False, segwit=True, parent_name=None, safe_index=-1):
        """
        public_key_list: List of public keys. Must be in bytes.
        m: integer
        n: integer
        _privkey: PrivateKey object. If you provide a Private key, don't provide a master_privkey.
        master_pubkey: Xtended_pubkey object. If you provide a master_privkey, it is not necessary to
        provide a master_pubkey.
        master_privkey: Xtended_privkey object. If you provide a master_privkey, don't provide a _privkey.
        testnet: Boolean.
        segwit: Boolean.
        child_wallet: Boolean.
        """
        if isinstance(master_pubkey, str):
            master_pubkey = Xtended_pubkey.parse(master_pubkey)
        
        if m < 1 or m > 16 or n < 1 or n > 16: raise Exception("m and n must be between 1 and 16")
        if m > n: raise Exception("m must be always less or equal than n")   
        if len(public_key_list) != (n-1):
            msg = "n and the amount of public keys don't match. The amount of public keys must always be n-1."
            raise Exception(msg)
        
        self.index = None
        #getting the index of the private key in the list of public keys
        if _privkey is not None and master_privkey is None and master_pubkey is not None:
            self.wallet_type = "simple"
            self.master_privkey = None
            self.privkey = _privkey
            self.pubkey = PrivateKey(int.from_bytes(self.privkey,"big")).point.sec()
            for i,public_key in enumerate(public_key_list):
                if public_key == self.pubkey:
                    self.index = i
            if self.index is None: raise Exception ("Private key must be able to generate one of the public keys.")
            self.master_pubkey = master_pubkey
        
        elif _privkey is None and master_privkey is not None:
            self.privkey = _privkey
            self.wallet_type = "main"
            self.master_privkey = master_privkey
            self.master_pubkey = self.master_privkey.xtended_public_key
            
        elif _privkey is not None and master_privkey is not None:
            msg1 = "Provide either a Private key or a master_privkey, not both. Provide a _privkey if you are not "
            msg2 = "the main creator of the wallet or if you received this wallet as an invitation. Provide a "
            msg3 = "master_privkey if you have the public keys of the cosigners and you are in charge of sharing "
            msg4 = "this wallet with the cosigners."
            raise Exception (msg1+msg2+msg3+msg4)
            
        elif _privkey is None and master_privkey is None and master_pubkey is not None: 
            self.wallet_type = "watch-only"
            self.privkey = _privkey
            self.master_privkey = None
            self.master_pubkey = master_pubkey
        else:
            raise Exception ("You can only skip providing a master public key when you provide a master private key.")
        
        self.public_key_list = public_key_list
        self.m = m
        self.n = n
        self.addr_type = addr_type
        self.testnet = testnet
        self.segwit =segwit
        self.name = name
        self.parent_name = parent_name
        self.safe_index = safe_index
        
        #save the new wallet in the database
        #but first, lets convert the public keys to bytes to avoid problems in the database
        int_pubkeys = [int.from_bytes(x,"big") for x in self.public_key_list]
        if self.privkey is not None: int_privkey = int.from_bytes(self.privkey,"big")
        else: int_privkey = None
        if self.master_privkey is None: xpriv = None
        else: xpriv = str(self.master_privkey.xtended_key)
        
        self.start_conn()
        self.db.new_SHDSafeWallet(self.name, str(int_pubkeys), self.m, self.n, str(int_privkey),
                                  xpriv, str(self.master_pubkey.get_xtended_key()),
                                  self.addr_type, int(self.testnet), int(self.segwit), self.parent_name, 
                                  self.safe_index)
        print(f"self: {self}")
        self.close_conn()
        
    def __repr__(cls):
        return f"SHDSafeWallet xpub:{cls.master_pubkey}, pubkeys: {cls.public_key_list}"
        
    @classmethod    
    def from_privkey_masterpubkey(cls, name, public_key_list, master_pubkey, _privkey, m=2, n=6,addr_type="p2wsh", 
                                   testnet=False, segwit=True,parent_name=None,index=-1):
        """
        simple.
        From a private key that is not the master private key. 
        """
        return cls(name,public_key_list=public_key_list, m=m, n=n,addr_type=addr_type,_privkey=_privkey, 
                   master_pubkey=master_pubkey,testnet=testnet, segwit=segwit,parent_name=parent_name, 
                   safe_index=index)
    
    @classmethod    
    def from_master_privkey(cls, name, public_key_list, master_privkey, m=2, n=6,addr_type="p2wsh",
                            testnet=False, segwit=True,  parent_name=None,index=-1):
        """
        main
        """
        return cls(name,public_key_list=public_key_list, m=m, n=n, addr_type=addr_type, master_privkey=master_privkey, 
                   testnet=testnet, segwit=segwit,parent_name=parent_name, safe_index=index)
    
    @classmethod    
    def watch_only(cls, name, public_key_list,master_pubkey,m=2,n=6,addr_type="p2wsh",testnet=False, segwit=True, 
                    parent_name=None, index=-1):
        """
        watch-only
        """
        return cls(name,public_key_list=public_key_list, m=m, n=n, addr_type=addr_type, master_pubkey=master_pubkey,
                   testnet=testnet, segwit=segwit, parent_name=parent_name, safe_index=index)
    
    @classmethod   
    def from_database(self, name):
        self.start_conn(self)
        w = self.db.recover_SHDSafeWallet(name)
        try: w = w[0]
        except: 
            print(f"wallet with name {name} not found ")
            return False
        self.close_conn(self)
        print(w)
        
        
        pubkeys = w[1][1:-1].split(", ")
        public_key_list = [int(x).to_bytes(33,"big") for x in pubkeys]
        print(public_key_list)
        if w[4] == "None": privkey = None
        else: privkey = int(w[4]).to_bytes(32,"big")
        if w[6] == "None": master_pubkey=None 
        else: master_pubkey = Xtended_pubkey.parse(w[6])
        if w[5] == "None": master_privkey = None
        else: master_privkey = Xtended_privkey.parse(w[5])
        if w[10] == "None": parent_name = None
        else: parent_name = w[10]
        
        return self(name, public_key_list, w[2],w[3],w[7],privkey,master_pubkey,master_privkey,
                    w[8],w[9],parent_name,w[11])
    
    def get_i(self,wallet_name, account_path, index):
        
        self.start_conn()

        last_index = self.db.get_max_index( account_path, wallet_name)

        if last_index is None: i = 0
        else: i = last_index + 1
        
        self.close_conn()
        return i
        
    def get_child_wallet(self, index, path):
        """
        This is the wallet for specific daily or weekly safes. Therefore, the index must comply with the format YYMMDD
        for daily safes, and YYWW for weekly safes.
        index: Integer.
        """
        #data validation
        if self.safe_index > -1: raise Exception ("Only master wallets can create child wallets.")  
        if index < 2000 or index > 999999 or (index > 9999 and index < 200101):
            raise Exception ("Bad index for daily or weekly safe. Follow YYMMDD for daily or YYWW for weekly.")
        
        full_path = path + str(index)
        #wallet creation depending on the type of wallet
        if self.wallet_type == "main":
            child_xtended_privkey = self.master_privkey.get_child_from_path(full_path)
            
            child_wallet = SHDSafeWallet.from_master_privkey( str(index)+"_"+self.name,self.public_key_list, 
                                                             child_xtended_privkey, 
                                                             self.m,self.n,self.addr_type,self.testnet, self.segwit,
                                                            self.name, index)
        elif self.wallet_type == "simple":
            child_xtended_pubkey = self.master_pubkey.get_child_from_path(full_path)
            
            child_wallet = SHDSafeWallet.from_privkey_masterpubkey(str(index)+"_"+self.name,self.public_key_list, 
                                                                   child_xtended_pubkey,
                                                                   self.privkey, self.m, self.n,self.addr_type, 
                                                                   self.testnet, self.segwit,self.name, index)
        elif self.wallet_type == "watch-only":
            child_xtended_pubkey = self.master_pubkey.get_child_from_path(full_path)
            
            child_wallet = SHDSafeWallet.watch_only(str(index)+"_"+self.name,self.public_key_list,child_xtended_pubkey,
                                                    self.m,self.n,
                                                    self.addr_type,self.testnet, self.segwit,self.name, index)
        
        return child_wallet
    
    def get_daily_safe_wallet(self, index=None):
        """
        Returns a daily_safe wallet based on the index.
        If today's wallet is desired, don't pass any argument or set index to None.
        """
        if self.safe_index >= 0: raise Exception ("Only master wallets can create safe wallets.")
        path = "m/"
        if index is None:
            today = datetime.datetime.now()
            index_string = str(today.year)[2:]+'{:02d}'.format(today.month)+'{:02d}'.format(today.day)
            index = int(index_string)
            
        safe_wallet = self.get_child_wallet(index, path)
        
        return safe_wallet
    
    
    def get_unused_addresses_list(self, change_addresses=False):
        
        self.start_conn()
        unused_addresses = self.db.get_unused_safe_addresses(name = self.name)
        self.close_conn()
        print(f"unused_addresses: {unused_addresses}")
        
        if    change_addresses: unused_addresses_filtered =  [x for x in unused_addresses if x[1]==1]
        else: unused_addresses_filtered =  [x for x in unused_addresses if x[1]==0]
        
        return unused_addresses_filtered
                               
                               
    def create_receiving_address(self, addr_type = "p2wsh",index=0):
        if self.safe_index < 0: raise Exception ("Only child wallets can create addresses.")
        if addr_type.lower() ==    "p2sh":       _type  = P2SH
        elif addr_type.lower() ==   "p2wsh":     _type  = P2WSH
        elif addr_type.lower() == "p2sh_p2wsh":  _type  = P2SH_P2WSH
        else: raise Exception(f"{addr_type} is NOT a valid type of address.")
        receiving_path = "m/0/"
        #i = self.get_i(self.name,receiving_path, index)
        i = index
        path = receiving_path + str(i)
        print(f"Deposit address's Path: {path}")
        
        if self.privkey is not None and self.master_privkey is None: 
            
            privkey = PrivateKey(int.from_bytes(self.privkey,"big"))
            
            account = SHMAccount(m=self.m, n=self.n, xtended_pubkey = str(self.master_pubkey), 
                                 public_key_list = self.public_key_list,  _privkey=privkey,
                                 addr_type=self.addr_type, testnet=self.testnet, segwit=self.segwit)
            
        elif self.privkey is None and self.master_privkey is not None:
            
            account = SHMAccount(m=self.m, n=self.n, xtended_pubkey = str(self.master_pubkey), 
                                 public_key_list = self.public_key_list,  master_privkey=self.master_privkey,
                                 addr_type=self.addr_type, testnet=self.testnet, segwit=self.segwit)
            
        else: privkey = None
        
        
        
        address = account.get_deposit_address(i)
        self.start_conn()
        self.db.new_address(address,receiving_path,i,FALSE, _type, self.name, self.safe_index)
        self.close_conn()
        return address

    
    def create_change_address(self, addr_type = "p2wsh", index=None):
        if self.safe_index < 0: raise Exception ("Only child wallets can create addresses.")
        receiving_path = "m/1/"
        
        i = self.get_i(self.name,receiving_path, index)
        path = receiving_path + str(i)
        print(f"Change address's Path: {path}")
        
        if self.privkey is not None and self.master_privkey is None: 
            
            privkey = PrivateKey(int.from_bytes(self.privkey,"big"))
            
            account = SHMAccount(m=self.m, n=self.n, xtended_pubkey = str(self.master_pubkey), 
                                 public_key_list = self.public_key_list,  _privkey=privkey,
                                 addr_type=self.addr_type, testnet=self.testnet, segwit=self.segwit)
            
        elif self.privkey is None and self.master_privkey is not None:
            
            account = SHMAccount(m=self.m, n=self.n, xtended_pubkey = str(self.master_pubkey), 
                                 public_key_list = self.public_key_list,  master_privkey=self.master_privkey,
                                 addr_type=self.addr_type, testnet=self.testnet, segwit=self.segwit)
            
        else: privkey = None
        
        #address = account.get_change_address(index=i)
        change_account = account.get_change_account(index=i)
        self.start_conn()
        self.db.new_address(change_account.address,receiving_path,i,TRUE, P2WSH, self.name, self.safe_index)
        self.close_conn()
        return change_account
    
    def get_a_change_address(self):
        """
        Returns an Account object containing a change address. It returns an existing unused change
        address account, or creates a new one if necessary.
        """
        if self.safe_index < 0: raise Exception ("Only child wallets can create addresses.")
        self.start_conn()
        unused_addresses = self.get_unused_addresses_list(change_addresses=True)
        self.close_conn()
        
        if len(unused_addresses)>0: return self.create_change_address(index=unused_addresses[0][3])
        else: return self.create_change_address()
        
    def share(self):
        if self.safe_index >= 0: raise Exception ("Only master wallets can be shared.")
        return {"public_key_list":self.public_key_list,"m":self.m, "n":self.n, "addr_type":self.addr_type,
                "master_pubkey":self.master_pubkey,"testnet":self.testnet, "segwit":self.segwit }
    

    def get_utxos(self):
        """
        This method is an overwritten version of the original Wallet-class version.
        """
        self.start_conn()
        ################################ FOR DEVOLPMENT PORPUSES ONLY ###############################
        ############### CHANGE BACK TO ORIGINAL STATE ONCE DONE WITH TESTS ##########################
        #coins = self.db.look_for_coins(self.name) #ORIGINAL LINE!
        coins = self.db.look_for_coins("200827_test1_master_wallet") #TEST LINE
        ############################################################################################
        self.close_conn()
        return coins
    
    def update_balance(self):
        """
        This method is an overwritten version of the original Wallet-class version.
        """
        self.start_conn()
        addresses = self.db.get_all_addresses(self.name)
        self.close_conn()
        
        if self.testnet: coin_symbol = "btc-testnet"
        else: coin_symbol = "btc"
            
        for address in addresses:
            "address will be a touple with data (address,)"
            print(f"consulting blockchain for address {address}")
            addr_info = get_address_details(address[0], coin_symbol = coin_symbol, unspent_only=True)
            print(f"res for {address}:\n{addr_info}")
            
            self.start_conn()
            if addr_info["unconfirmed_n_tx"] > 0:
                print("got unconfirmed transactions.")
                for utxo in addr_info["unconfirmed_txrefs"]:
                    if not self.db.exist_utxo( utxo["tx_hash"], utxo["tx_output_n"], 0):
                        print("new unconfirmed UTXO")
                        self.db.new_utxo(utxo["address"],utxo["value"],utxo["tx_hash"],utxo["tx_output_n"],confirmed = 0)
            
            if addr_info["n_tx"] - addr_info["unconfirmed_n_tx"] > 0 :
                print("got transactions.")
                for utxo in addr_info["txrefs"]:
                    if not self.db.exist_utxo( utxo["tx_hash"], utxo["tx_output_n"], 1):
                        print("new confirmed UTXO")
                        self.db.new_utxo(addr_info["address"],utxo["value"],utxo["tx_hash"],utxo["tx_output_n"],
                                         confirmed = 1)
      
        self.close_conn()
        return self.get_balance()
        
    def get_coins(self, to_address_amount_list, segwit=True):
        send_all = False
        
        print(f"From wallet.send(): {to_address_amount_list}")
        #calculate total amount to send from the wallet
        total_amount = 0
        for output in to_address_amount_list:
            total_amount += output[1]
            
            
        #we validate that we have funds to carry out the transaction, or if we are trying to send all the funds  
        balance = self.get_balance()
        if total_amount>balance:
            raise Exception(f"Not enough funds in wallet for this transaction.\nOnly {balance} satoshis available")
        
        elif total_amount == balance: send_all = True
            
            
        #We choose the utxos to spend for the transaction
        all_utxos = self.get_utxos()
        
        #If we are trying to send all the funds, then we choose all the utxos
        if send_all: utxos = all_utxos
        #If not, then we have to choose in an efficient manner.
        else:    
            utxos = []
            utxo_total = 0
            
            #First we check if only one utxo can carry out the whole transaction
            for utxo in all_utxos:
                if utxo[2] > total_amount*1.1:
                    utxos = [utxo]
                    break
            #if we found that none of the utxos is big enough to carry out the whole tx, we have to choose several.
            if len(utxos)==0:    
                for utxo in all_utxos:
                    utxos.append(utxo)
                    utxo_total += utxo[2]
                    if utxo_total > (total_amount*1.1) : break
        
        return utxos, send_all
    
    def build_tx(self, utxos, to_address_amount_list, segwit=True, send_all = False):
        
        account = self.get_signing_account()
        
        #if we want to send all the funds we don't need a change address.
        if send_all: change_address = None    
        else:        change_address = self.get_a_change_address()
        """    
        privkey = PrivateKey(int.from_bytes(self.privkey,"big"))
            
        account = SHMAccount(self.m, self.n, str(self.master_pubkey), self.public_key_list, privkey,
                             self.addr_type, self.testnet, self.segwit)
        """
        #We create the Multi-Signature Transaction object
        tx_response = MultiSigTransaction.create_from_wallet(utxo_list=utxos,
                                                             receivingAddress_w_amount_list =to_address_amount_list,
                                                             multi_sig_account = account,change_address = change_address,
                                                             fee=None,segwit=True,send_all=send_all)
        
        return tx_response
    
    
    def get_signing_account(self):
        
        #we take care of validating and formatting the private key
        if self.privkey is not None and self.master_privkey is None: 
            
            privkey = PrivateKey(int.from_bytes(self.privkey,"big"))
            
            account = SHMAccount(m=self.m, n=self.n, xtended_pubkey = str(self.master_pubkey), 
                                 public_key_list = self.public_key_list,  _privkey=privkey,
                                 addr_type=self.addr_type, testnet=self.testnet, segwit=self.segwit)
            
        elif self.privkey is None and self.master_privkey is not None:
            
            account = SHMAccount(m=self.m, n=self.n, xtended_pubkey = str(self.master_pubkey), 
                                 public_key_list = self.public_key_list,  master_privkey=self.master_privkey,
                                 addr_type=self.addr_type, testnet=self.testnet, segwit=self.segwit)
            
        else: privkey = None
        #else: raise Exception("Multi signature Transaction with a master key is not developed yet.")
        
        #We create the multisignature account to sign the transaction. This only works for spending from deposit
        #account. To spend from the change account, we need to do something else.
        
        
        
        #account = shm_account.get_deposit_account()
        return account
        
        
    def broadcast_tx(self, tx,utxos):
        
        if self.testnet: symbol = "btc-testnet"
        else: symbol= "btc"

        env = Sqlite3Environment()
        BLOCKCYPHER_API_KEY = env.get_key("BLOCKCYPHER_API_KEY")[0][0]
        env.close_database()
        push = pushtx(tx_hex= tx.serialize().hex(), coin_symbol=symbol,api_key=BLOCKCYPHER_API_KEY)
        tx_id = tx.id()
        print(f"TRANSACTION ID: {tx_id}")

        #saving in db
        self.start_conn()
        self.db.new_tx(tx_id, [ (x[0],x[1]) for x in utxos] ,
                       [str(x).split(":")+[i] for i,x in enumerate(tx.tx_outs)])
        self.close_conn()
        #start new thread to check if the transaction goes through in the blockchain or not.
        #Maybe move this to the front end class to pop a warning when it doesn't go through
        mythread = threading.Thread(target=self.confirm_tx_sent,args=(tx_id,))
        mythread.start()

        return tx,push
        
        
    def send(self, to_address_amount_list, segwit=True):
        """
        to_address can be a single address or a list.
        amount can be an integer or a list of integers.
        If they are lists, they must be ordered in the same way. address 1 will e sent amount 1, 
        adrress 2 will be sent amount2.. adress n will be sent amount n.
        """
        #First let's get ready the inputs for the transaction
        utxos, send_all = self.get_coins(to_address_amount_list, segwit)
        
        #Now, let's build the transaction
        tx_response = self.build_tx(utxos,to_address_amount_list, segwit, send_all)
        
        # tx_response will be a touple of the transaction object and a boolean (tx,READY) that tells us if the  
        #transaction is ready to be broadcasted or if it needs more signatures:
        if tx_response[1]: 
            #if it is ready, we broadcast the transaction
            self.broadcast_tx(tx_response[0],utxos)
            
        return tx_response 
        
    def sign_received_tx(self, tx):
        """
        tx: Tx object. The transaction object that needs to be signed.
        """
        
        #to do: we need to get the utxo list from the database using the transaction to be able to sign
        utxos = []
        self.start_conn()
        for _in in tx.tx_ins:
            res = self.db.get_utxo_info(_in.prev_tx.hex(), _in.prev_index)
            print(res)
            utxos.append(res[0])
            
        self.close_conn()
        #We create the multisignature account to sign the transaction.
        account = self.get_signing_account()
        
        tx_response = MultiSigTransaction.sign_received_tx_with_wallet(utxos,tx,account)
        
        if tx_response[1]: self.broadcast_tx(tx_response[0],utxos)
            
        return tx_response
            
            
            

In [107]:
from constants import *
from wallet import Wallet
from accounts import SHMAccount, MultSigAccount
from ecc import PrivateKey, S256Point
from transactions import MultiSigTransaction
import datetime
from bip32 import Xtended_pubkey, Xtended_privkey
from blockcypher import get_address_details, get_transaction_details
from blockcypher import pushtx
import threading
from wallet_database_sqlite3 import Sqlite3Wallet, Sqlite3Environment

class HDMWallet(Wallet):
    """
    SafeWallet is a Multisignature SHD wallet.
    SHDM Single Herarchical Deterministic Multi-Signature Wallet
    """
    def __init__(self, name, master_pubkey_list,master_privkey, m=2, n=6,addr_type="p2wsh",  
                  testnet=False, segwit=True, parent_name=None, safe_index=-1):
        """
        public_key_list: List of public keys. Must be in bytes.
        m: integer
        n: integer
        _privkey: PrivateKey object. If you provide a Private key, don't provide a master_privkey.
        master_pubkey: Xtended_pubkey object. If you provide a master_privkey, it is not necessary to
        provide a master_pubkey.
        master_privkey: Xtended_privkey object. If you provide a master_privkey, don't provide a _privkey.
        testnet: Boolean.
        segwit: Boolean.
        child_wallet: Boolean.
        """
        
        if m < 1 or m > 16 or n < 1 or n > 16:
            raise Exception("m and n must be between 1 and 16")
        if m > n:
            raise Exception("m must be always less or equal than n")
        if len(master_pubkey_list) != (n):
            raise Exception("n and the amount of public keys don't match. The amount of public keys must always be n-1.")
        
        
        index = -1
        #getting the index of the private key in the list of public keys
        self.master_privkey = master_privkey
        pubkey = str(self.master_privkey.xtended_public_key)
        for i,public_key in enumerate(master_pubkey_list):
            if str(public_key) == pubkey:
                index = i
        if index < 0: raise Exception ("Private key must correspond to one of the public keys.")

        self.xtended_pubkey_list = master_pubkey_list
        self.name = name
        self.m = m
        self.n = n
        self.testnet = testnet
        self.addr_type = addr_type.lower()
        self.privkey_index = index
        self.parent_name = parent_name
        self.safe_index = safe_index
        self.segwit = segwit
        
        #save the new wallet in the database
        #but first, lets convert the public keys into strings to avoid problems in the database
        str_pubkeys = [str(x.get_xtended_key()) for x in self.xtended_pubkey_list]
        xpriv = str(self.master_privkey.xtended_key)
        
        self.start_conn()
        self.db.new_HDMWallet(self.name, str_pubkeys, self.m, self.n,xpriv, self.addr_type, int(self.testnet), 
                                  int(self.segwit), self.parent_name, self.safe_index)
        self.close_conn()
        
    def __repr__(cls):
        return f"FHDSafeWallet :{cls.m}-of-{cls.n} pubkeys: {cls.xtended_pubkey_list}"
    
    @classmethod   
    def from_database(self, name):
        self.start_conn(self)
        w = self.db.recover_HDMWallet(name)
        try: w = w[0]
        except: 
            print(f"wallet with name {name} not found ")
            return False
        self.close_conn(self)
        print(w)
        pubkeys = w[1][1:-1].split(", ")
        master_pubkey_list = [Xtended_pubkey.parse(x) for x in pubkeys]
        print(f"parsed master public keys: {public_key_list}")
        master_privkey = Xtended_privkey.parse(w[4])
        if w[8] == "None": parent_name = None
        else: parent_name = w[8]
        
        return self(name, master_pubkey_list,master_privkey, w[2],w[3],
                    w[5],w[6],w[7],parent_name,w[9])
    
    def get_i(self,wallet_name, account_path, index):
        
        self.start_conn()

        last_index = self.db.get_max_index( account_path, wallet_name)

        if last_index is None: i = 0
        else: i = last_index + 1
        
        self.close_conn()
        return i
    
    def get_child_wallet(self, index, path):
        """
        This is the wallet for specific years. Therefore, the index must comply with the format YY.
        index: Integer.
        """
        #data validation
        if self.safe_index > -1: raise Exception ("Only master wallets can create child wallets.")  
        if index < 20 or index > 99:
            raise Exception ("Bad index for year wallet. Follow YY format.")
        
        full_path = path + str(index)
        child_xtended_privkey = self.master_privkey.get_child_from_path(full_path)
        
        child_pubkey_list = [x.get_child_from_path(full_path) for x in self.xtended_pubkey_list]
            
        child_wallet = HDMWallet( "20"+str(index)+"_"+self.name,  child_pubkey_list,
                                 child_xtended_privkey,  self.m,  self.n,  self.addr_type,
                                 self.testnet, self.segwit,  self.name, index)
        
        return child_wallet
    
    def get_year_wallet(self, index=None):
        """
        Returns a year_safe wallet based on the index.
        If this-years's wallet is desired, don't pass any argument or set index to None.
        """
        if self.safe_index >= 0: raise Exception ("Only master wallets can create safe wallets.")
        path = "m/"
        if index is None:
            today = datetime.datetime.now()
            index_string = str(today.year)[2:]
            index = int(index_string)
            
        safe_wallet = self.get_child_wallet(index, path)
        
        return safe_wallet
    
    def get_unused_addresses_list(self, change_addresses=False):
        
        self.start_conn()
        unused_addresses = self.db.get_unused_safe_addresses(name = self.name)
        self.close_conn()
        print(f"unused_addresses: {unused_addresses}")
        
        if    change_addresses: unused_addresses_filtered =  [x for x in unused_addresses if x[1]==1]
        else: unused_addresses_filtered =  [x for x in unused_addresses if x[1]==0]
        
        return unused_addresses_filtered
                               
                               
    def create_receiving_address(self, addr_type = "p2wsh",index=None):
        if self.safe_index < 0: raise Exception ("Only child wallets can create addresses.")
        if addr_type.lower() ==    "p2sh":       _type  = P2SH
        elif addr_type.lower() ==   "p2wsh":     _type  = P2WSH
        elif addr_type.lower() == "p2sh_p2wsh":  _type  = P2SH_P2WSH
        else: raise Exception(f"{addr_type} is NOT a valid type of address.")
        receiving_path = "m/0/"
        i = self.get_i(self.name,receiving_path, index)
        path = receiving_path + str(i)
        print(f"Deposit address's Path: {path}")
        
        account = FHMAccount(self.m, self.xtended_pubkey_list, self.master_privkey, self.addr_type,
                             self.testnet, self.segwit)
        
        
        
        address = account.get_deposit_address(i)
        self.start_conn()
        self.db.new_address(address,receiving_path,i,FALSE, _type, self.name, self.safe_index)
        self.close_conn()
        return address

    
    def create_change_address(self, addr_type = "p2wsh", index=None):
        if self.safe_index < 0: raise Exception ("Only child wallets can create addresses.")
        receiving_path = "m/1/"
        
        i = self.get_i(self.name,receiving_path, index)
        path = receiving_path + str(i)
        print(f"Change address's Path: {path}")
        
        account = FHMAccount(self.m, self.xtended_pubkey_list, self.master_privkey, self.addr_type,
                             self.testnet, self.segwit)
        
        change_account = account.get_change_account(index=i)
        self.start_conn()
        self.db.new_address(change_account.address,receiving_path,i,TRUE, P2WSH, self.name, self.safe_index)
        self.close_conn()
        return change_account
    
    def get_a_change_address(self):
        """
        Returns an Account object containing a change address. It returns an existing unused change
        address account, or creates a new one if necessary.
        """
        if self.safe_index < 0: raise Exception ("Only child wallets can create addresses.")
        self.start_conn()
        unused_addresses = self.get_unused_addresses_list(change_addresses=True)
        self.close_conn()
        
        if len(unused_addresses)>0: return self.create_change_address(index=unused_addresses[0][3])
        else: return self.create_change_address()
        
    def share(self):
        if self.safe_index >= 0: raise Exception ("Only master wallets can be shared.")
        return {"master_pubkey_list":self.xtended_pubkey_list,"m":self.m, "n":self.n, "addr_type":self.addr_type,
                "testnet":self.testnet, "segwit":self.segwit }
    

    def get_utxos(self):
        """
        This method is an overwritten version of the original Wallet-class version.
        """
        self.start_conn()
        ################################ FOR DEVOLPMENT PORPUSES ONLY ###############################
        ############### CHANGE BACK TO ORIGINAL STATE ONCE DONE WITH TESTS ##########################
        #coins = self.db.look_for_coins(self.name) #ORIGINAL LINE!
        coins = self.db.look_for_coins("2020_Corporate_wallet_master") #TEST LINE
        ############################################################################################
        self.close_conn()
        return coins
    
    def update_balance(self):
        """
        This method is an overwritten version of the original Wallet-class version.
        """
        self.start_conn()
        addresses = self.db.get_all_addresses(self.name)
        self.close_conn()
        
        if self.testnet: coin_symbol = "btc-testnet"
        else: coin_symbol = "btc"
            
        for address in addresses:
            "address will be a touple with data (address,)"
            print(f"consulting blockchain for address {address}")
            addr_info = get_address_details(address[0], coin_symbol = coin_symbol, unspent_only=True)
            print(f"res for {address}:\n{addr_info}")
            
            self.start_conn()
            if addr_info["unconfirmed_n_tx"] > 0:
                print("got unconfirmed transactions.")
                for utxo in addr_info["unconfirmed_txrefs"]:
                    if not self.db.exist_utxo( utxo["tx_hash"], utxo["tx_output_n"], 0):
                        print("new unconfirmed UTXO")
                        self.db.new_utxo(utxo["address"],utxo["value"],utxo["tx_hash"],utxo["tx_output_n"],confirmed = 0)
            
            if addr_info["n_tx"] - addr_info["unconfirmed_n_tx"] > 0 :
                print("got transactions.")
                for utxo in addr_info["txrefs"]:
                    if not self.db.exist_utxo( utxo["tx_hash"], utxo["tx_output_n"], 1):
                        print("new confirmed UTXO")
                        self.db.new_utxo(addr_info["address"],utxo["value"],utxo["tx_hash"],utxo["tx_output_n"],
                                         confirmed = 1)
      
        self.close_conn()
        return self.get_balance()
        
    def get_coins(self, to_address_amount_list, segwit=True):
        send_all = False
        
        print(f"From wallet.send(): {to_address_amount_list}")
        #calculate total amount to send from the wallet
        total_amount = 0
        for output in to_address_amount_list:
            total_amount += output[1]
            
            
        #we validate that we have funds to carry out the transaction, or if we are trying to send all the funds  
        balance = self.get_balance()
        if total_amount>balance:
            raise Exception(f"Not enough funds in wallet for this transaction.\nOnly {balance} satoshis available")
        
        elif total_amount == balance: send_all = True
            
            
        #We choose the utxos to spend for the transaction
        all_utxos = self.get_utxos()
        
        #If we are trying to send all the funds, then we choose all the utxos
        if send_all: utxos = all_utxos
        #If not, then we have to choose in an efficient manner.
        else:    
            utxos = []
            utxo_total = 0
            
            #First we check if only one utxo can carry out the whole transaction
            for utxo in all_utxos:
                if utxo[2] > total_amount*1.1:
                    utxos = [utxo]
                    break
            #if we found that none of the utxos is big enough to carry out the whole tx, we have to choose several.
            if len(utxos)==0:    
                for utxo in all_utxos:
                    utxos.append(utxo)
                    utxo_total += utxo[2]
                    if utxo_total > (total_amount*1.1) : break
        
        return utxos, send_all
    
    def build_tx(self, utxos, to_address_amount_list, segwit=True, send_all = False):
        
        account = self.get_signing_account()
        
        #if we want to send all the funds we don't need a change address.
        if send_all: change_address = None    
        else:        change_address = self.get_a_change_address()
        
        #We create the Multi-Signature Transaction object
        tx_response = MultiSigTransaction.create_from_wallet(utxo_list=utxos,
                                                             receivingAddress_w_amount_list =to_address_amount_list,
                                                             multi_sig_account = account,change_address = change_address,
                                                             fee=None,segwit=True,send_all=send_all)
        
        return tx_response
    
    
    def get_signing_account(self):
        
        #We create the multisignature account to sign the transaction. 
        account = FHMAccount(self.m, self.xtended_pubkey_list, self.master_privkey, self.addr_type,
                             self.testnet, self.segwit)
        
        return account
        
        
    def broadcast_tx(self, tx,utxos):
        
        if self.testnet: symbol = "btc-testnet"
        else: symbol= "btc"

        env = Sqlite3Environment()
        BLOCKCYPHER_API_KEY = env.get_key("BLOCKCYPHER_API_KEY")[0][0]
        env.close_database()
        push = pushtx(tx_hex= tx.serialize().hex(), coin_symbol=symbol,api_key=BLOCKCYPHER_API_KEY)
        tx_id = tx.id()
        print(f"TRANSACTION ID: {tx_id}")

        #saving in db
        self.start_conn()
        self.db.new_tx(tx_id, [ (x[0],x[1]) for x in utxos] ,
                       [str(x).split(":")+[i] for i,x in enumerate(tx.tx_outs)])
        self.close_conn()
        #start new thread to check if the transaction goes through in the blockchain or not.
        #Maybe move this to the front end class to pop a warning when it doesn't go through
        mythread = threading.Thread(target=self.confirm_tx_sent,args=(tx_id,))
        mythread.start()

        return tx,push
        
        
    def send(self, to_address_amount_list, segwit=True):
        """
        to_address can be a single address or a list.
        amount can be an integer or a list of integers.
        If they are lists, they must be ordered in the same way. address 1 will e sent amount 1, 
        adrress 2 will be sent amount2.. adress n will be sent amount n.
        """
        #First let's get ready the inputs for the transaction
        utxos, send_all = self.get_coins(to_address_amount_list, segwit)
        
        #Now, let's build the transaction
        tx_response = self.build_tx(utxos,to_address_amount_list, segwit, send_all)
        
        # tx_response will be a touple of the transaction object and a boolean (tx,READY) that tells us if the  
        #transaction is ready to be broadcasted or if it needs more signatures:
        if tx_response[1]: 
            #if it is ready, we broadcast the transaction
            self.broadcast_tx(tx_response[0],utxos)
            
        return tx_response 
        
    def sign_received_tx(self, tx):
        """
        tx: Tx object. The transaction object that needs to be signed.
        """
        
        #to do: we need to get the utxo list from the database using the transaction to be able to sign
        utxos = []
        self.start_conn()
        for _in in tx.tx_ins:
            res = self.db.get_utxo_info(_in.prev_tx.hex(), _in.prev_index)
            print(res)
            utxos.append(res[0])
            
        self.close_conn()
        #We create the multisignature account to sign the transaction.
        account = self.get_signing_account()
        
        tx_response = MultiSigTransaction.sign_received_tx_with_wallet(utxos,tx,account)
        
        if tx_response[1]: self.broadcast_tx(tx_response[0],utxos)
            
        return tx_response
            

In [7]:
words = "engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning"
manager1 = ManagerWallet.recover_from_words(words, 256, "RobertPauslon",True)

words = "client sudden sunset borrow pupil rely sand girl prefer movie bachelor guilt giraffe glove much strong dizzy switch ill silent goddess crumble goat power"
manager2 = ManagerWallet.recover_from_words(words,entropy=256,passphrase="RobertPaulson",testnet=True)


public_keys = [ manager1.get_daily_safe_pubkey(), manager2.get_daily_safe_pubkey(), PrivateKey(int.from_bytes(b"Ashley Serna","little")).point.sec()]


words_master="other seek spend wise milk govern conduct crunch train wrestle twist cattle"
master_wallet = CorporateSuperWallet.recover_from_words(words_master,128,"",True)

8418572761394767459444418608530371567217744336876557229650655736270882421023962287566834689847418048023007033142570964655978481626857402470209587761075589
6018716354834777059445516500463325153454936853151215624488541646839939367247233986759821493097378851875115552924255831805062990606698886137563075847103799
7349618075688827342644292161792411154388397071333136974711210419399617419860687939173705085036785308619307965881391603570352784472532296452988784284606559


In [8]:
"""
main_wallet = SHDSafeWallet.from_master_privkey("test1_master_wallet",public_keys,
                                                master_privkey=master_wallet.get_child_from_path("m/44H/0H/1H"),
                                               n=4,testnet=True)
                                               """

'\nmain_wallet = SHDSafeWallet.from_master_privkey("test1_master_wallet",public_keys,\n                                                master_privkey=master_wallet.get_child_from_path("m/44H/0H/1H"),\n                                               n=4,testnet=True)\n                                               '

In [8]:
main_wallet = SHDSafeWallet.from_database("test1_master_wallet")

connection with database made.
('test1_master_wallet', '[461222389963770672910023206337944803706847222730930779040024025202330161562314, 242181816824935352779949905339585673702570459364503992825384375679130606102141, 349045194032852365244858435810860057589910251399394147451364362662197641799889]', 2, 4, 'None', 'tprv8fVXQGqBdhmo7Lymx7JARMiyt59rg9JuuS5uxVsgqKCpQt5wSD9G6HY8uVQs5CPNV5i6qt1vWes78DmmbrKa1gncZXpaL3m71GT3RsRcWda', 'tpubDCBZYgsRn5TTzp1ZqkxkpmP6T6fnqUVpUjghF1uzFb1DFNLi4bxrGnA15c4fLY14bbpZnT1uhNqZPmZHY2yNNFfGJTSAxHZgVj6yDUGSThv', 'p2wsh', 1, 1, 'None', -1)
[b'\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\xc2\xca', b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}", b'\x03\x03\xb0\x94-\xbe\xd2\xd0Nq\xab+\xa9\xbc\x05\xb9 \xf8O\xfe+\x8b\x8c-\x8c\xfd\x80*\xda`\xf6\xe8\xd1']
connection with database made.
INSERT OR IGNORE INTO SHDSafeWallet (name,public_key_list,m,n,privkey,xpriv,xpub,ad

In [9]:
todays_wallet = main_wallet.get_daily_safe_wallet()

connection with database made.
INSERT OR IGNORE INTO SHDSafeWallet (name,public_key_list,m,n,privkey,xpriv,xpub,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("200827_test1_master_wallet","[461222389963770672910023206337944803706847222730930779040024025202330161562314, 242181816824935352779949905339585673702570459364503992825384375679130606102141, 349045194032852365244858435810860057589910251399394147451364362662197641799889]",2,4,"None","tprv8iHAxPu9bvwpwVXXZdFXLcb8JyUP4g5y73AWkzURfSoAozGdqrF8457WxGfX5oHoDRdChWGNS259giKbcmDTHuSwRUuR8N3cBvHgHHRMgSC","tpubDEyD6owPkJdVpxZKTGv7k2FEszzKE1GsgLmJ3WWj5ibZeUXQUF4iEZjP8Q4eX8aiQoScEcu8b8WMWNpqkWnbwMhZVsrtLza3eHehefNvwrZ","p2wsh",1,1,'test1_master_wallet',200827) ;
self: SHDSafeWallet xpub:tpubDEyD6owPkJdVpxZKTGv7k2FEszzKE1GsgLmJ3WWj5ibZeUXQUF4iEZjP8Q4eX8aiQoScEcu8b8WMWNpqkWnbwMhZVsrtLza3eHehefNvwrZ, pubkeys: [b'\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\xc2\xca', b"\x02\x17n\x0c\xfd\x84]'\

In [10]:
todays_address = todays_wallet.create_receiving_address()
todays_address

Deposit address's Path: m/0/0
ini shm account _privkey: None
connection with database made.
INSERT OR IGNORE INTO Addresses (address, path, acc_index, change_addr, created, type, wallet, safe_index)
 VALUES("tb1qdns32eklqlkzwzr54pvddw5t0m28j080j5tkstysuw26cspd3whsx9xqd5", "m/0/", 0, 0, 1598552179, 5, "200827_test1_master_wallet", 200827) ;


'tb1qdns32eklqlkzwzr54pvddw5t0m28j080j5tkstysuw26cspd3whsx9xqd5'

In [11]:
invite = main_wallet.share()
invite

{'public_key_list': [b'\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\xc2\xca',
  b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}",
  b'\x03\x03\xb0\x94-\xbe\xd2\xd0Nq\xab+\xa9\xbc\x05\xb9 \xf8O\xfe+\x8b\x8c-\x8c\xfd\x80*\xda`\xf6\xe8\xd1'],
 'm': 2,
 'n': 4,
 'addr_type': 'p2wsh',
 'master_pubkey': tpubDCBZYgsRn5TTzp1ZqkxkpmP6T6fnqUVpUjghF1uzFb1DFNLi4bxrGnA15c4fLY14bbpZnT1uhNqZPmZHY2yNNFfGJTSAxHZgVj6yDUGSThv,
 'testnet': 1,
 'segwit': 1}

In [12]:
manager_safe_wallet = manager1.accept_invite(invite,"manager1_daily_safe")

b'\xd3\xcdG\xa0\xa1\xbf\x86V\x879\xa2\x05Q3o\xb7(\x92Z\x19\xc8bzf\xf7\x9e\xd4\xfb\xc9\xf5\xbdP'
connection with database made.
INSERT OR IGNORE INTO SHDSafeWallet (name,public_key_list,m,n,privkey,xpriv,xpub,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("manager1_daily_safe","[461222389963770672910023206337944803706847222730930779040024025202330161562314, 242181816824935352779949905339585673702570459364503992825384375679130606102141, 349045194032852365244858435810860057589910251399394147451364362662197641799889]",2,4,"95800709053964933282561433899801232546169819566735397106442210103505108450640","None","tpubDCBZYgsRn5TTzp1ZqkxkpmP6T6fnqUVpUjghF1uzFb1DFNLi4bxrGnA15c4fLY14bbpZnT1uhNqZPmZHY2yNNFfGJTSAxHZgVj6yDUGSThv","p2wsh",1,1,'None',-1) ;
self: SHDSafeWallet xpub:tpubDCBZYgsRn5TTzp1ZqkxkpmP6T6fnqUVpUjghF1uzFb1DFNLi4bxrGnA15c4fLY14bbpZnT1uhNqZPmZHY2yNNFfGJTSAxHZgVj6yDUGSThv, pubkeys: [b'\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\

In [13]:

todays_managers_wallet = manager_safe_wallet.get_daily_safe_wallet()

connection with database made.
INSERT OR IGNORE INTO SHDSafeWallet (name,public_key_list,m,n,privkey,xpriv,xpub,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("200827_manager1_daily_safe","[461222389963770672910023206337944803706847222730930779040024025202330161562314, 242181816824935352779949905339585673702570459364503992825384375679130606102141, 349045194032852365244858435810860057589910251399394147451364362662197641799889]",2,4,"95800709053964933282561433899801232546169819566735397106442210103505108450640","None","tpubDEyD6owPkJdVpxZKTGv7k2FEszzKE1GsgLmJ3WWj5ibZeUXQUF4iEZjP8Q4eX8aiQoScEcu8b8WMWNpqkWnbwMhZVsrtLza3eHehefNvwrZ","p2wsh",1,1,'manager1_daily_safe',200827) ;
self: SHDSafeWallet xpub:tpubDEyD6owPkJdVpxZKTGv7k2FEszzKE1GsgLmJ3WWj5ibZeUXQUF4iEZjP8Q4eX8aiQoScEcu8b8WMWNpqkWnbwMhZVsrtLza3eHehefNvwrZ, pubkeys: [b'\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\xc2\xca', b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x

In [14]:
todays_managers_wallet.create_receiving_address()

Deposit address's Path: m/0/0
ini shm account _privkey: <ecc.PrivateKey object at 0x7f8ae77d49d0>
connection with database made.
INSERT OR IGNORE INTO Addresses (address, path, acc_index, change_addr, created, type, wallet, safe_index)
 VALUES("tb1qdns32eklqlkzwzr54pvddw5t0m28j080j5tkstysuw26cspd3whsx9xqd5", "m/0/", 0, 0, 1598552195, 5, "200827_manager1_daily_safe", 200827) ;


'tb1qdns32eklqlkzwzr54pvddw5t0m28j080j5tkstysuw26cspd3whsx9xqd5'

In [15]:
manager2_safe_wallet = manager2.accept_invite(invite,"manager_daily_safe")

b"\xfe\x8e\xa2X\xe1\xe0\x9b\xef\xd2s\x1f;\xd5\xf3g\x88\xfd\x97\x13C/\xe3H\xdf\xb8E\xcc[$H\x1f'"
connection with database made.
INSERT OR IGNORE INTO SHDSafeWallet (name,public_key_list,m,n,privkey,xpriv,xpub,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("manager_daily_safe","[461222389963770672910023206337944803706847222730930779040024025202330161562314, 242181816824935352779949905339585673702570459364503992825384375679130606102141, 349045194032852365244858435810860057589910251399394147451364362662197641799889]",2,4,"115139476302519344463634772255798084337463636322414000002775671153459500424999","None","tpubDCBZYgsRn5TTzp1ZqkxkpmP6T6fnqUVpUjghF1uzFb1DFNLi4bxrGnA15c4fLY14bbpZnT1uhNqZPmZHY2yNNFfGJTSAxHZgVj6yDUGSThv","p2wsh",1,1,'None',-1) ;
self: SHDSafeWallet xpub:tpubDCBZYgsRn5TTzp1ZqkxkpmP6T6fnqUVpUjghF1uzFb1DFNLi4bxrGnA15c4fLY14bbpZnT1uhNqZPmZHY2yNNFfGJTSAxHZgVj6yDUGSThv, pubkeys: [b'\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\

In [16]:
todays_managers2_wallet = manager2_safe_wallet.get_daily_safe_wallet()

connection with database made.
INSERT OR IGNORE INTO SHDSafeWallet (name,public_key_list,m,n,privkey,xpriv,xpub,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("200827_manager_daily_safe","[461222389963770672910023206337944803706847222730930779040024025202330161562314, 242181816824935352779949905339585673702570459364503992825384375679130606102141, 349045194032852365244858435810860057589910251399394147451364362662197641799889]",2,4,"115139476302519344463634772255798084337463636322414000002775671153459500424999","None","tpubDEyD6owPkJdVpxZKTGv7k2FEszzKE1GsgLmJ3WWj5ibZeUXQUF4iEZjP8Q4eX8aiQoScEcu8b8WMWNpqkWnbwMhZVsrtLza3eHehefNvwrZ","p2wsh",1,1,'manager_daily_safe',200827) ;
self: SHDSafeWallet xpub:tpubDEyD6owPkJdVpxZKTGv7k2FEszzKE1GsgLmJ3WWj5ibZeUXQUF4iEZjP8Q4eX8aiQoScEcu8b8WMWNpqkWnbwMhZVsrtLza3eHehefNvwrZ, pubkeys: [b'\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\xc2\xca', b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x9

In [17]:
todays_managers2_wallet.create_receiving_address()

Deposit address's Path: m/0/0
ini shm account _privkey: <ecc.PrivateKey object at 0x7f8ae78aa4d0>
connection with database made.
INSERT OR IGNORE INTO Addresses (address, path, acc_index, change_addr, created, type, wallet, safe_index)
 VALUES("tb1qdns32eklqlkzwzr54pvddw5t0m28j080j5tkstysuw26cspd3whsx9xqd5", "m/0/", 0, 0, 1598552209, 5, "200827_manager_daily_safe", 200827) ;


'tb1qdns32eklqlkzwzr54pvddw5t0m28j080j5tkstysuw26cspd3whsx9xqd5'

In [18]:
todays_wallet.update_balance()

connection with database made.
consulting blockchain for address ('tb1qdns32eklqlkzwzr54pvddw5t0m28j080j5tkstysuw26cspd3whsx9xqd5',)
res for ('tb1qdns32eklqlkzwzr54pvddw5t0m28j080j5tkstysuw26cspd3whsx9xqd5',):
{'address': 'tb1qdns32eklqlkzwzr54pvddw5t0m28j080j5tkstysuw26cspd3whsx9xqd5', 'total_received': 200000, 'total_sent': 100000, 'balance': 100000, 'unconfirmed_balance': 0, 'final_balance': 100000, 'n_tx': 3, 'unconfirmed_n_tx': 0, 'final_n_tx': 3, 'txrefs': [{'tx_hash': 'f9613cbef74bb3d6dc12afcec13e2b61d20361d5541329f33b3e4f86648da986', 'block_height': 1827914, 'tx_input_n': -1, 'tx_output_n': 0, 'value': 100000, 'ref_balance': 100000, 'spent': False, 'confirmations': 130, 'confirmed': datetime.datetime(2020, 8, 27, 17, 31, 30, tzinfo=tzutc()), 'double_spend': False}], 'tx_url': 'https://api.blockcypher.com/v1/btc/test3/txs/', 'unconfirmed_txrefs': []}
connection with database made.
got transactions.
SELECT * FROM Utxos WHERE  tx_id = 'f9613cbef74bb3d6dc12afcec13e2b61d20361d554132

100000

In [19]:
m = [("mkHS9ne12qx9pS9VojpwU5xtRd4T7X7ZUt",50000)]
response = todays_wallet.send(m)

From wallet.send(): [('mkHS9ne12qx9pS9VojpwU5xtRd4T7X7ZUt', 50000)]
connection with database made.
[('f9613cbef74bb3d6dc12afcec13e2b61d20361d5541329f33b3e4f86648da986', 0, 100000, 'm/0/', 0, 'tb1qdns32eklqlkzwzr54pvddw5t0m28j080j5tkstysuw26cspd3whsx9xqd5', 5)]
('f9613cbef74bb3d6dc12afcec13e2b61d20361d5541329f33b3e4f86648da986', 0, 100000, 'm/0/', 0, 'tb1qdns32eklqlkzwzr54pvddw5t0m28j080j5tkstysuw26cspd3whsx9xqd5', 5)
connection with database made.
ini shm account _privkey: None
connection with database made.
connection with database made.
SELECT address, change_addr, path, acc_index  
FROM Addresses WHERE
NOT EXISTS(
SELECT 1 
 FROM Utxos
WHERE Utxos.address = Addresses.address) 
AND
 Addresses.wallet = '200827_test1_master_wallet';
unused_addresses: [('tb1qqa8j2l955lgxfme4nt663cvqtr6v26ujeumdxq3ld7psy367x90q0nq3mc', 1, 'm/1/', 0)]
connection with database made.
Change address's Path: m/1/1
ini shm account _privkey: None
connection with database made.
INSERT OR IGNORE INTO Addresses (a

In [20]:
tx = todays_managers2_wallet.sign_received_tx(response[0])

connection with database made.
SELECT Utxos.tx_id, Utxos.out_index, Utxos.amount, Addresses.path, Addresses.acc_index, Addresses.address,
 Addresses.type 
FROM Utxos INNER JOIN Addresses 
ON Utxos.address = Addresses.address
WHERE Utxos.tx_id = 'f9613cbef74bb3d6dc12afcec13e2b61d20361d5541329f33b3e4f86648da986' AND Utxos.out_index = 0;
[('f9613cbef74bb3d6dc12afcec13e2b61d20361d5541329f33b3e4f86648da986', 0, 100000, 'm/0/', 0, 'tb1qdns32eklqlkzwzr54pvddw5t0m28j080j5tkstysuw26cspd3whsx9xqd5', 5)]
ini shm account _privkey: <ecc.PrivateKey object at 0x7f8ae7278710>
Address trying to spend from: tb1qdns32eklqlkzwzr54pvddw5t0m28j080j5tkstysuw26cspd3whsx9xqd5, path m/0/0
Deposit account index 0
tb1qdns32eklqlkzwzr54pvddw5t0m28j080j5tkstysuw26cspd3whsx9xqd5
sign_input_multisig commands: [0, b"0D\x02 '\xed\xcf 2\xe3\xe1\xa95\xd6\xd4\xbd\xa2\xacL?\xcdp!\x84\x98\xd7\xe3 5\xdbo~\xf2\x81\xb7J\x02 6g\xd9C\xf7\x8c\xc2\xcdJ-D\xa2\x0b\x8bF \xf8\x8c\x905\xe5\x9b\xa0E\xe78m\xba\x11@SQ\x01", 0, b'0E\x02!\x

# Transaction from an SHDSafeWallet succesful!

d4a11095338e4e4e7e7a3a1c61a5ec63aa7de230155ab6c34dcd8e6a5c51fb14

## Now let's test FHDMWallets

In [36]:
words = "engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning"
corporate1 = CorporateSuperWallet.recover_from_words(words, 256, "RobertPauslon",True)

words = "client sudden sunset borrow pupil rely sand girl prefer movie bachelor guilt giraffe glove much strong dizzy switch ill silent goddess crumble goat power"
corporate2 = CorporateSuperWallet.recover_from_words(words,entropy=256,passphrase="RobertPaulson",testnet=True)


words_master="other seek spend wise milk govern conduct crunch train wrestle twist cattle"
corporate_master = CorporateSuperWallet.recover_from_words(words_master,128,"",True)

public_keys = [ corporate1.get_year_xpub(), corporate2.get_year_xpub(), corporate_master.get_year_xpub()]

public_keys

8418572761394767459444418608530371567217744336876557229650655736270882421023962287566834689847418048023007033142570964655978481626857402470209587761075589
6018716354834777059445516500463325153454936853151215624488541646839939367247233986759821493097378851875115552924255831805062990606698886137563075847103799
7349618075688827342644292161792411154388397071333136974711210419399617419860687939173705085036785308619307965881391603570352784472532296452988784284606559


[tpubDCYEqvE5F87BQzZovqjWAtH1xo2ViiqBLCEEkWqd5zN2UBfHo2qKL1UTw1vawzyRffohXZn5tdgxGXJu1MzNHBfoByCm9YotHswF8uQ8qyq,
 tpubDDJvTw8uvAEBhJLHyVd2mpzb6AxYMSwLcRgNcHbHLjshksYw3z7U7o3VkxNa9RJ7EnwvrEXmrkCGF8kq5xMTMkaVVWdtxvwbnGQNDhBJcGS,
 tpubDCBZYgsRn5TU4ntpt6ne9hMwYyHVV4NHyuumRPwgWNrTJ7CvHEiZ175Y9UfvKtgL271U54ABuuAbxmZ5X9gMT2g7SoETaxP3tG5LJogennJ]

In [37]:
corporate_safe_master_wallet = HDMWallet("Corporate_wallet_master", public_keys,corporate_master.get_child_from_path("m/44H/0H/3H"), 
                              m=2, n=3,addr_type="p2wsh",  testnet=True, segwit=True)


connection with database made.
INSERT OR IGNORE INTO HDMWallet (name,master_pubkey_list,m,n,xpriv,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("Corporate_wallet_master","['tpubDCYEqvE5F87BQzZovqjWAtH1xo2ViiqBLCEEkWqd5zN2UBfHo2qKL1UTw1vawzyRffohXZn5tdgxGXJu1MzNHBfoByCm9YotHswF8uQ8qyq', 'tpubDDJvTw8uvAEBhJLHyVd2mpzb6AxYMSwLcRgNcHbHLjshksYw3z7U7o3VkxNa9RJ7EnwvrEXmrkCGF8kq5xMTMkaVVWdtxvwbnGQNDhBJcGS', 'tpubDCBZYgsRn5TU4ntpt6ne9hMwYyHVV4NHyuumRPwgWNrTJ7CvHEiZ175Y9UfvKtgL271U54ABuuAbxmZ5X9gMT2g7SoETaxP3tG5LJogennJ']",2,3,"tprv8fVXQGqBdhmoBKs2zT83kHhpywmZKjBPQcJz8suP6744Tcx9eqtxpcTfyMY3GEvM4ErgPrqPBL3gTzdNyKYsdLMU7ggGG5oR6KMc26i78gi","p2wsh",1,1,'None',-1) ;


In [38]:
this_year_safe_master_wallet = corporate_safe_master_wallet.get_year_wallet()

connection with database made.
INSERT OR IGNORE INTO HDMWallet (name,master_pubkey_list,m,n,xpriv,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("2020_Corporate_wallet_master","['tpubDFDEWUZ9ryeqPtMvRG5fhYYSntc5g14UW4QHx9dnndM8u7DCv2zvmMSsoNTPhLHjVvGcgLKgdzgb5E5xHnZn6wHSBENUW3JGmWq22z4LFtp', 'tpubDEP78B2RKZHgbWRxTrb5esRMAxderyoCEqvqyCXRS68bJDidDxCCmobm6Zf6smuUN2z2z1mb43JEiJFFUpAcVyR8CE3HRNDFAZnjCQScEwF', 'tpubDEaFyQfgCwFvSnJzZQUvN7hUejs4fa9itDdr2Ej3DTu28EnBtLes5cFZg1jk5bBgMbGSV7kwP3JQJJRRdVeXFirPuzEdyWue4HPjFcEd3rE']",2,3,"tprv8htDpzdS4ZaFZKHCfkpKxi3N5iM8WExpJv34jigjoC6dHkXRFwqGu7dhVswrY2L8s7xLbmpWCji2bscquKFh2RLGb34CBiccnoPV3DNACFj","p2wsh",1,1,'Corporate_wallet_master',20) ;


In [39]:
invite = corporate_safe_master_wallet.share()

In [40]:
invite

{'master_pubkey_list': [tpubDCYEqvE5F87BQzZovqjWAtH1xo2ViiqBLCEEkWqd5zN2UBfHo2qKL1UTw1vawzyRffohXZn5tdgxGXJu1MzNHBfoByCm9YotHswF8uQ8qyq,
  tpubDDJvTw8uvAEBhJLHyVd2mpzb6AxYMSwLcRgNcHbHLjshksYw3z7U7o3VkxNa9RJ7EnwvrEXmrkCGF8kq5xMTMkaVVWdtxvwbnGQNDhBJcGS,
  tpubDCBZYgsRn5TU4ntpt6ne9hMwYyHVV4NHyuumRPwgWNrTJ7CvHEiZ175Y9UfvKtgL271U54ABuuAbxmZ5X9gMT2g7SoETaxP3tG5LJogennJ],
 'm': 2,
 'n': 3,
 'addr_type': 'p2wsh',
 'testnet': True,
 'segwit': True}

In [41]:
corporate1_safe_master = corporate1.accept_corporate_safe_invite(invite,"Corporate1_safe_wallet")

accept_safe_wallet_invite: priv: tprv8frChWBq6kRWXXY23C4umUcuPmWZZPeGktdTTzoKfiZddhQXAe1j9WrbkrFhZnQ86my8oT5KSvLCev1WaC1BC6mGgqs7q6mSgeW6KZiXAqd; pubkey: {year_safe_acc.xtended_public_key}
connection with database made.
INSERT OR IGNORE INTO HDMWallet (name,master_pubkey_list,m,n,xpriv,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("Corporate1_safe_wallet","['tpubDCYEqvE5F87BQzZovqjWAtH1xo2ViiqBLCEEkWqd5zN2UBfHo2qKL1UTw1vawzyRffohXZn5tdgxGXJu1MzNHBfoByCm9YotHswF8uQ8qyq', 'tpubDDJvTw8uvAEBhJLHyVd2mpzb6AxYMSwLcRgNcHbHLjshksYw3z7U7o3VkxNa9RJ7EnwvrEXmrkCGF8kq5xMTMkaVVWdtxvwbnGQNDhBJcGS', 'tpubDCBZYgsRn5TU4ntpt6ne9hMwYyHVV4NHyuumRPwgWNrTJ7CvHEiZ175Y9UfvKtgL271U54ABuuAbxmZ5X9gMT2g7SoETaxP3tG5LJogennJ']",2,3,"tprv8frChWBq6kRWXXY23C4umUcuPmWZZPeGktdTTzoKfiZddhQXAe1j9WrbkrFhZnQ86my8oT5KSvLCev1WaC1BC6mGgqs7q6mSgeW6KZiXAqd","p2wsh",1,1,'None',-1) ;


In [42]:
corporate2_safe_master = corporate2.accept_corporate_safe_invite(invite,"Corporate2_safe_wallet")

accept_safe_wallet_invite: priv: tprv8gctKX6fmnYWoqJW5qxSNRLUX9ScC7kS385bKmYyvU5JvPJARbHswJRdaoPnzXgHdSo5GnQvMj6QFKmiYFGqwrU4aZSwPPPuJnrYNYgxvDj; pubkey: {year_safe_acc.xtended_public_key}
connection with database made.
INSERT OR IGNORE INTO HDMWallet (name,master_pubkey_list,m,n,xpriv,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("Corporate2_safe_wallet","['tpubDCYEqvE5F87BQzZovqjWAtH1xo2ViiqBLCEEkWqd5zN2UBfHo2qKL1UTw1vawzyRffohXZn5tdgxGXJu1MzNHBfoByCm9YotHswF8uQ8qyq', 'tpubDDJvTw8uvAEBhJLHyVd2mpzb6AxYMSwLcRgNcHbHLjshksYw3z7U7o3VkxNa9RJ7EnwvrEXmrkCGF8kq5xMTMkaVVWdtxvwbnGQNDhBJcGS', 'tpubDCBZYgsRn5TU4ntpt6ne9hMwYyHVV4NHyuumRPwgWNrTJ7CvHEiZ175Y9UfvKtgL271U54ABuuAbxmZ5X9gMT2g7SoETaxP3tG5LJogennJ']",2,3,"tprv8gctKX6fmnYWoqJW5qxSNRLUX9ScC7kS385bKmYyvU5JvPJARbHswJRdaoPnzXgHdSo5GnQvMj6QFKmiYFGqwrU4aZSwPPPuJnrYNYgxvDj","p2wsh",1,1,'None',-1) ;


In [43]:
this_year_safe_corporate1 = corporate1_safe_master.get_year_wallet()

connection with database made.
INSERT OR IGNORE INTO HDMWallet (name,master_pubkey_list,m,n,xpriv,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("2020_Corporate1_safe_wallet","['tpubDFDEWUZ9ryeqPtMvRG5fhYYSntc5g14UW4QHx9dnndM8u7DCv2zvmMSsoNTPhLHjVvGcgLKgdzgb5E5xHnZn6wHSBENUW3JGmWq22z4LFtp', 'tpubDEP78B2RKZHgbWRxTrb5esRMAxderyoCEqvqyCXRS68bJDidDxCCmobm6Zf6smuUN2z2z1mb43JEiJFFUpAcVyR8CE3HRNDFAZnjCQScEwF', 'tpubDEaFyQfgCwFvSnJzZQUvN7hUejs4fa9itDdr2Ej3DTu28EnBtLes5cFZg1jk5bBgMbGSV7kwP3JQJJRRdVeXFirPuzEdyWue4HPjFcEd3rE']",2,3,"tprv8iXCN4WuibyAWRL8XcR5J8tLDs69WfsZvkoWfdbVNMYk4cxSHeBLarq1dF5Qo1iKDhfiDmfEHw4uNPJPj15NF1BLXjg6PSLJKbc7GCGyoZj","p2wsh",1,1,'Corporate1_safe_wallet',20) ;


In [44]:
this_year_safe_corporate2 = corporate2_safe_master.get_year_wallet()

connection with database made.
INSERT OR IGNORE INTO HDMWallet (name,master_pubkey_list,m,n,xpriv,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("2020_Corporate2_safe_wallet","['tpubDFDEWUZ9ryeqPtMvRG5fhYYSntc5g14UW4QHx9dnndM8u7DCv2zvmMSsoNTPhLHjVvGcgLKgdzgb5E5xHnZn6wHSBENUW3JGmWq22z4LFtp', 'tpubDEP78B2RKZHgbWRxTrb5esRMAxderyoCEqvqyCXRS68bJDidDxCCmobm6Zf6smuUN2z2z1mb43JEiJFFUpAcVyR8CE3HRNDFAZnjCQScEwF', 'tpubDEaFyQfgCwFvSnJzZQUvN7hUejs4fa9itDdr2Ej3DTu28EnBtLes5cFZg1jk5bBgMbGSV7kwP3JQJJRRdVeXFirPuzEdyWue4HPjFcEd3rE']",2,3,"tprv8hh4ykzBBBc1i3QAaCvVFTmEbw7ihecHfYL4ggV81pLCTjTrbZNcbJytvRp9LcosVhUKWUCzSmoZh7mF3KyJaKy5NFyR2wgnRLpVsXhix7o","p2wsh",1,1,'Corporate2_safe_wallet',20) ;


In [45]:
address = this_year_safe_master_wallet.create_receiving_address()

connection with database made.
Deposit address's Path: m/0/5
public key 0 in list: tpubDFDEWUZ9ryeqPtMvRG5fhYYSntc5g14UW4QHx9dnndM8u7DCv2zvmMSsoNTPhLHjVvGcgLKgdzgb5E5xHnZn6wHSBENUW3JGmWq22z4LFtp
public key 1 in list: tpubDEP78B2RKZHgbWRxTrb5esRMAxderyoCEqvqyCXRS68bJDidDxCCmobm6Zf6smuUN2z2z1mb43JEiJFFUpAcVyR8CE3HRNDFAZnjCQScEwF
public key 2 in list: tpubDEaFyQfgCwFvSnJzZQUvN7hUejs4fa9itDdr2Ej3DTu28EnBtLes5cFZg1jk5bBgMbGSV7kwP3JQJJRRdVeXFirPuzEdyWue4HPjFcEd3rE
connection with database made.
INSERT OR IGNORE INTO Addresses (address, path, acc_index, change_addr, created, type, wallet, safe_index)
 VALUES("tb1qexedmz9lp7k3apt6yfz4g2zx908vc9husngk9l63mmns5l0l6jtqjpgtpr", "m/0/", 5, 0, 1598589093, 5, "2020_Corporate_wallet_master", 20) ;


In [46]:
addresss = this_year_safe_corporate1.create_receiving_address()

connection with database made.
Deposit address's Path: m/0/0
public key 0 in list: tpubDFDEWUZ9ryeqPtMvRG5fhYYSntc5g14UW4QHx9dnndM8u7DCv2zvmMSsoNTPhLHjVvGcgLKgdzgb5E5xHnZn6wHSBENUW3JGmWq22z4LFtp
public key 1 in list: tpubDEP78B2RKZHgbWRxTrb5esRMAxderyoCEqvqyCXRS68bJDidDxCCmobm6Zf6smuUN2z2z1mb43JEiJFFUpAcVyR8CE3HRNDFAZnjCQScEwF
public key 2 in list: tpubDEaFyQfgCwFvSnJzZQUvN7hUejs4fa9itDdr2Ej3DTu28EnBtLes5cFZg1jk5bBgMbGSV7kwP3JQJJRRdVeXFirPuzEdyWue4HPjFcEd3rE
connection with database made.
INSERT OR IGNORE INTO Addresses (address, path, acc_index, change_addr, created, type, wallet, safe_index)
 VALUES("tb1qs2ttef2l793njqx5eaa8evejc8vgr9nwjndrs9dat2n33mzea2tqedhsnx", "m/0/", 0, 0, 1598589093, 5, "2020_Corporate1_safe_wallet", 20) ;


In [47]:
addressss = this_year_safe_corporate2.create_receiving_address()

connection with database made.
Deposit address's Path: m/0/0
public key 0 in list: tpubDFDEWUZ9ryeqPtMvRG5fhYYSntc5g14UW4QHx9dnndM8u7DCv2zvmMSsoNTPhLHjVvGcgLKgdzgb5E5xHnZn6wHSBENUW3JGmWq22z4LFtp
public key 1 in list: tpubDEP78B2RKZHgbWRxTrb5esRMAxderyoCEqvqyCXRS68bJDidDxCCmobm6Zf6smuUN2z2z1mb43JEiJFFUpAcVyR8CE3HRNDFAZnjCQScEwF
public key 2 in list: tpubDEaFyQfgCwFvSnJzZQUvN7hUejs4fa9itDdr2Ej3DTu28EnBtLes5cFZg1jk5bBgMbGSV7kwP3JQJJRRdVeXFirPuzEdyWue4HPjFcEd3rE
connection with database made.
INSERT OR IGNORE INTO Addresses (address, path, acc_index, change_addr, created, type, wallet, safe_index)
 VALUES("tb1qs2ttef2l793njqx5eaa8evejc8vgr9nwjndrs9dat2n33mzea2tqedhsnx", "m/0/", 0, 0, 1598589094, 5, "2020_Corporate2_safe_wallet", 20) ;


In [48]:
this_year_safe_master_wallet.update_balance()

connection with database made.
consulting blockchain for address ('tb1q8u9pzx94wr0cxu337tc7g8hz6tza72j7h6znsdk4j2jmxl3dpe0qgzkcj3',)
res for ('tb1q8u9pzx94wr0cxu337tc7g8hz6tza72j7h6znsdk4j2jmxl3dpe0qgzkcj3',):
{'address': 'tb1q8u9pzx94wr0cxu337tc7g8hz6tza72j7h6znsdk4j2jmxl3dpe0qgzkcj3', 'total_received': 0, 'total_sent': 0, 'balance': 0, 'unconfirmed_balance': 0, 'final_balance': 0, 'n_tx': 0, 'unconfirmed_n_tx': 0, 'final_n_tx': 0, 'tx_url': 'https://api.blockcypher.com/v1/btc/test3/txs/', 'txrefs': [], 'unconfirmed_txrefs': []}
connection with database made.
consulting blockchain for address ('tb1q8whhqx28gd4yamsve66slnfav7j2rnycuwwzy8k8g3ym6tvlsshstt3hfs',)
res for ('tb1q8whhqx28gd4yamsve66slnfav7j2rnycuwwzy8k8g3ym6tvlsshstt3hfs',):
{'address': 'tb1q8whhqx28gd4yamsve66slnfav7j2rnycuwwzy8k8g3ym6tvlsshstt3hfs', 'total_received': 0, 'total_sent': 0, 'balance': 0, 'unconfirmed_balance': 0, 'final_balance': 0, 'n_tx': 0, 'unconfirmed_n_tx': 0, 'final_n_tx': 0, 'tx_url': 'https://api.bloc

100000

In [49]:
m = [("mkHS9ne12qx9pS9VojpwU5xtRd4T7X7ZUt",50000)]
response = this_year_safe_master_wallet.send(m)

From wallet.send(): [('mkHS9ne12qx9pS9VojpwU5xtRd4T7X7ZUt', 50000)]
connection with database made.
[('127c852465639c19bda8cfd9a6786a30e00053e1bbc4fc9c9aaf623b32919b38', 0, 100000, 'm/0/', 0, 'tb1qs2ttef2l793njqx5eaa8evejc8vgr9nwjndrs9dat2n33mzea2tqedhsnx', 5)]
('127c852465639c19bda8cfd9a6786a30e00053e1bbc4fc9c9aaf623b32919b38', 0, 100000, 'm/0/', 0, 'tb1qs2ttef2l793njqx5eaa8evejc8vgr9nwjndrs9dat2n33mzea2tqedhsnx', 5)
connection with database made.
public key 0 in list: tpubDFDEWUZ9ryeqPtMvRG5fhYYSntc5g14UW4QHx9dnndM8u7DCv2zvmMSsoNTPhLHjVvGcgLKgdzgb5E5xHnZn6wHSBENUW3JGmWq22z4LFtp
public key 1 in list: tpubDEP78B2RKZHgbWRxTrb5esRMAxderyoCEqvqyCXRS68bJDidDxCCmobm6Zf6smuUN2z2z1mb43JEiJFFUpAcVyR8CE3HRNDFAZnjCQScEwF
public key 2 in list: tpubDEaFyQfgCwFvSnJzZQUvN7hUejs4fa9itDdr2Ej3DTu28EnBtLes5cFZg1jk5bBgMbGSV7kwP3JQJJRRdVeXFirPuzEdyWue4HPjFcEd3rE
connection with database made.
connection with database made.
SELECT address, change_addr, path, acc_index  
FROM Addresses WHERE
NOT EXISTS(
SELE

In [50]:
tx = this_year_safe_corporate1.sign_received_tx(response[0])

connection with database made.
SELECT Utxos.tx_id, Utxos.out_index, Utxos.amount, Addresses.path, Addresses.acc_index, Addresses.address,
 Addresses.type 
FROM Utxos INNER JOIN Addresses 
ON Utxos.address = Addresses.address
WHERE Utxos.tx_id = '127c852465639c19bda8cfd9a6786a30e00053e1bbc4fc9c9aaf623b32919b38' AND Utxos.out_index = 0;
[('127c852465639c19bda8cfd9a6786a30e00053e1bbc4fc9c9aaf623b32919b38', 0, 100000, 'm/0/', 0, 'tb1qs2ttef2l793njqx5eaa8evejc8vgr9nwjndrs9dat2n33mzea2tqedhsnx', 5)]
public key 0 in list: tpubDFDEWUZ9ryeqPtMvRG5fhYYSntc5g14UW4QHx9dnndM8u7DCv2zvmMSsoNTPhLHjVvGcgLKgdzgb5E5xHnZn6wHSBENUW3JGmWq22z4LFtp
public key 1 in list: tpubDEP78B2RKZHgbWRxTrb5esRMAxderyoCEqvqyCXRS68bJDidDxCCmobm6Zf6smuUN2z2z1mb43JEiJFFUpAcVyR8CE3HRNDFAZnjCQScEwF
public key 2 in list: tpubDEaFyQfgCwFvSnJzZQUvN7hUejs4fa9itDdr2Ej3DTu28EnBtLes5cFZg1jk5bBgMbGSV7kwP3JQJJRRdVeXFirPuzEdyWue4HPjFcEd3rE
Address trying to spend from: tb1qs2ttef2l793njqx5eaa8evejc8vgr9nwjndrs9dat2n33mzea2tqedhsnx, path 

In [58]:
he = int.from_bytes(b"222d2deda24c03b2cc2b458c35780c62b6b5603c088709f17607b7ecf1399e4d7","big")

In [59]:
type(he)

int

In [62]:
he.to_bytes(66,"big")

b'\x00222d2deda24c03b2cc2b458c35780c62b6b5603c088709f17607b7ecf1399e4d7'

In [31]:
a = [{"a":1},{"B":2},{"cd":4}]
b = [list(x.keys())[0] for x in a]

In [32]:
b

['a', 'B', 'cd']

In [33]:
c = [x for x in b if x.startswith("c")]

In [35]:
c

['cd']

In [7]:
" ".join(b[:2])

'a B'

In [36]:
a = [{"a":{"B":2,"cd":4}}]

In [37]:
a

[{'a': {'B': 2, 'cd': 4}}]

In [50]:
z=a[0]["a"]
z

{'B': 119, 'cd': 4, 'e': 9}

In [51]:
for key in z:
    print(z[key])

119
4
9


In [41]:
a

[{'a': {'B': 119, 'cd': 4, 'e': 9}}]

In [108]:
words = "engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning"
manager1 = ManagerWallet.recover_from_words(words, 256, "RobertPauslon",True)

words = "client sudden sunset borrow pupil rely sand girl prefer movie bachelor guilt giraffe glove much strong dizzy switch ill silent goddess crumble goat power"
manager2 = ManagerWallet.recover_from_words(words,entropy=256,passphrase="RobertPaulson",testnet=True)

words = "aware pond potato mechanic market glue title addict chapter theory silly book"
manager3 = ManagerWallet.recover_from_words(words, 128, "",True)

words = "dry opera level brass sentence hamster spare beyond ethics floor gloom valid"
manager4 = CorporateSuperWallet.recover_from_words(words,entropy=128,passphrase="",testnet=True)

words = "ocean engage among slab laugh carry siege place meadow picnic huge refuse"
manager5 = CorporateSuperWallet.recover_from_words(words, 128, "",True)



8418572761394767459444418608530371567217744336876557229650655736270882421023962287566834689847418048023007033142570964655978481626857402470209587761075589
6018716354834777059445516500463325153454936853151215624488541646839939367247233986759821493097378851875115552924255831805062990606698886137563075847103799
4162409734528611111756301033770563963875662849627349706423163651536961341908306279907529065775660129270783374619650014334270481880624029028566693881161579
6909639706114781673414903746617077396660679925789934783580039639894650562409120041882686723903187546059433358926004652243464850825988704126919377509722019
9677066697512560492143972752022598285123803888462171670387143205574154293603133214186864151036559633064051743887438640912994313261160774593012036646074285


In [67]:
#manager3 = ManagerWallet.generate_random(testnet=True)

Copy these words for future recovery:
b'aware pond potato mechanic market glue title addict chapter theory silly book'
4162409734528611111756301033770563963875662849627349706423163651536961341908306279907529065775660129270783374619650014334270481880624029028566693881161579


In [68]:
#manager4 = CorporateSuperWallet.generate_random(testnet=True)

Copy these words for future recovery:
b'dry opera level brass sentence hamster spare beyond ethics floor gloom valid'
6909639706114781673414903746617077396660679925789934783580039639894650562409120041882686723903187546059433358926004652243464850825988704126919377509722019


In [69]:
#manager5 = CorporateSuperWallet.generate_random(testnet=True)

Copy these words for future recovery:
b'ocean engage among slab laugh carry siege place meadow picnic huge refuse'
9677066697512560492143972752022598285123803888462171670387143205574154293603133214186864151036559633064051743887438640912994313261160774593012036646074285


In [83]:
print(manager1.xtended_public_key,"\n",manager2.xtended_public_key,"\n",manager3.xtended_public_key,"\n",\
     manager4.xtended_public_key,"\n",manager5.xtended_public_key)


tpubD6NzVbkrYhZ4YsLLdYumHCp4fTyvvopGGP4uUvt8pkG5itXwubeeNbsjpmpBmHNZ2d3bKbXK7EPwUFZrpTU4Fb6GMuYGpr7Gweqvvg2UHuF 
 tpubD6NzVbkrYhZ4X2kKaAoh2fM3CuJyyDq3QaZNx78crQtQZNWme8kvzes6xriBHyE4CJ5uPHRWb6wzbKotkEPL5aFHexmPBZ94P3PSYyKwLWj 
 tpubD6NzVbkrYhZ4XTEtSoGcjUMRpvazsGSubezgBquk3SLGv6SzPoLiP6wKhgKdxM4mRxBE69ouh7sDnQWkyXCvGLsk59TPD9qbSwnn6eHGU1f 
 tpubD6NzVbkrYhZ4XNeSC1uJBoCYWrYk1FJLJjLyR5UujHij2UhtEqXqqfB6AvNCygoawR6MddXmAdbks43NFvivxXbjQmtVtZKAuqAC1p2pWAb 
 tpubD6NzVbkrYhZ4XnZYCe4vKN6MyNT1bWRtHZn3ChoCqSj5W8FHuSvYP8ZdscjZb96DVNQPcChoDMC5ZrTirtfRrTnjrV1ZQFA35pe3DepMrBD


In [85]:
"""
tpubD6NzVbkrYhZ4YsLLdYumHCp4fTyvvopGGP4uUvt8pkG5itXwubeeNbsjpmpBmHNZ2d3bKbXK7EPwUFZrpTU4Fb6GMuYGpr7Gweqvvg2UHuF 
 tpubD6NzVbkrYhZ4X2kKaAoh2fM3CuJyyDq3QaZNx78crQtQZNWme8kvzes6xriBHyE4CJ5uPHRWb6wzbKotkEPL5aFHexmPBZ94P3PSYyKwLWj 
 tpubD6NzVbkrYhZ4XTEtSoGcjUMRpvazsGSubezgBquk3SLGv6SzPoLiP6wKhgKdxM4mRxBE69ouh7sDnQWkyXCvGLsk59TPD9qbSwnn6eHGU1f 
 tpubD6NzVbkrYhZ4XNeSC1uJBoCYWrYk1FJLJjLyR5UujHij2UhtEqXqqfB6AvNCygoawR6MddXmAdbks43NFvivxXbjQmtVtZKAuqAC1p2pWAb 
 tpubD6NzVbkrYhZ4XnZYCe4vKN6MyNT1bWRtHZn3ChoCqSj5W8FHuSvYP8ZdscjZb96DVNQPcChoDMC5ZrTirtfRrTnjrV1ZQFA35pe3DepMrBD
"""

print(manager1.get_daily_safe_pubkey(),"\n",manager2.get_daily_safe_pubkey(),"\n",manager3.get_daily_safe_pubkey(),"\n",\
     manager4.get_daily_safe_pubkey(),"\n",manager5.get_daily_safe_pubkey())

b'\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\xc2\xca' 
 b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}" 
 b'\x02\x81\xf8\xdf;r\x17\x1f\xf1\xe2_\xef\xe2\x98\xba\xa2X\xac\xf6\xe7[/\xca\xads\x1b\x8d\x12D\xc8\xf7\x1aJ' 
 b'\x02\x1bP\xbbp\x89\xe5\xa2Q\x12\x1cp\x19\x03\xcd+\xc6Oa\xc7\x1a#\xaf\xcf$G\xa4\xed\x8bY\xa1\x14+' 
 b'\x03@\xb5e\x00\xfbN\xe7\x02\xa5\xa3\xa1K\x1c\xf4\x13$;r)\xb7\xe8\xa3\x02~\xf2|Q\xf6\x0cF\xa8\xb4'


In [86]:
print(int.from_bytes(manager1.get_daily_safe_pubkey(),"big"),
      "\n",int.from_bytes(manager2.get_daily_safe_pubkey(),"big"),
      "\n",int.from_bytes(manager3.get_daily_safe_pubkey(),"big"),
      "\n",int.from_bytes(manager4.get_daily_safe_pubkey(),"big"),
      "\n",int.from_bytes(manager5.get_daily_safe_pubkey(),"big")
     )

461222389963770672910023206337944803706847222730930779040024025202330161562314 
 242181816824935352779949905339585673702570459364503992825384375679130606102141 
 290372254706026096153481489592931453085780088683353495329679236536122868177482 
 243939266812165980563239760114596239186104406092581261489726331066308569928747 
 376644786442849392304182314585976636972636060283971456632342867625859504056500


In [89]:
t = 461222389963770672910023206337944803706847222730930779040024025202330161562314
print(t.to_bytes(33,"big"))
SHDSafeWallet

b'\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\xc2\xca'


In [88]:
t

"b'\\x03\\xfb\\xb2\\x9f(\\xe2:\\x8d\\x07\\xf0X68\\xfa\\x89\\xe8e\\xba\\xd0-\\xd1yk\\xfae\\xad\\x95\\xa5\\x17v*\\xc2\\xca'"

In [90]:
c= int("2222")
c

2222

In [91]:
this_week = datetime.datetime.now().isocalendar()

In [92]:
this_week

(2020, 46, 7)

In [93]:
this_week = datetime.datetime.now().isocalendar()
index_string = str(this_week[0])[2:]+str(this_week[1])
index = int(index_string)
index

2046

In [95]:
import calendar
calendar.month_abbr[1]

'Jan'

In [96]:
a= "ab"
b="abc do"
print(a in b)

True


In [109]:
invite ={'public_key_list': [b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}", b'\x02\x1bP\xbbp\x89\xe5\xa2Q\x12\x1cp\x19\x03\xcd+\xc6Oa\xc7\x1a#\xaf\xcf$G\xa4\xed\x8bY\xa1\x14+', b'\x02\x81\xf8\xdf;r\x17\x1f\xf1\xe2_\xef\xe2\x98\xba\xa2X\xac\xf6\xe7[/\xca\xads\x1b\x8d\x12D\xc8\xf7\x1aJ', b'\x03@\xb5e\x00\xfbN\xe7\x02\xa5\xa3\xa1K\x1c\xf4\x13$;r)\xb7\xe8\xa3\x02~\xf2|Q\xf6\x0cF\xa8\xb4'], 'm': 2, 'n': 5, 'addr_type': 'p2wsh', 'master_pubkey': "tpubDDPhf39HYJ34itcntagMVucwfUzvf6npJHGBMBuDaoUjyEeVFgvzgi8FbuQnRdMzMDd81kiqs7zcsGQRFC3B75QQQbVbtUM5cqnr7gWuqdf", 'testnet': 1, 'segwit': 1}

In [110]:
invite

{'public_key_list': [b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}",
  b'\x02\x1bP\xbbp\x89\xe5\xa2Q\x12\x1cp\x19\x03\xcd+\xc6Oa\xc7\x1a#\xaf\xcf$G\xa4\xed\x8bY\xa1\x14+',
  b'\x02\x81\xf8\xdf;r\x17\x1f\xf1\xe2_\xef\xe2\x98\xba\xa2X\xac\xf6\xe7[/\xca\xads\x1b\x8d\x12D\xc8\xf7\x1aJ',
  b'\x03@\xb5e\x00\xfbN\xe7\x02\xa5\xa3\xa1K\x1c\xf4\x13$;r)\xb7\xe8\xa3\x02~\xf2|Q\xf6\x0cF\xa8\xb4'],
 'm': 2,
 'n': 5,
 'addr_type': 'p2wsh',
 'master_pubkey': 'tpubDDPhf39HYJ34itcntagMVucwfUzvf6npJHGBMBuDaoUjyEeVFgvzgi8FbuQnRdMzMDd81kiqs7zcsGQRFC3B75QQQbVbtUM5cqnr7gWuqdf',
 'testnet': 1,
 'segwit': 1}

In [111]:
manager2_safe_wallet = manager2.accept_invite(invite,"test_store_safe_manager2")

b"\xfe\x8e\xa2X\xe1\xe0\x9b\xef\xd2s\x1f;\xd5\xf3g\x88\xfd\x97\x13C/\xe3H\xdf\xb8E\xcc[$H\x1f'"
connection with database made.
INSERT OR IGNORE INTO SHDSafeWallet (name,public_key_list,m,n,privkey,xpriv,xpub,addr_type,testnet,segwit,parent_name ,safe_index) 
VALUES("test_store_safe_manager2","[242181816824935352779949905339585673702570459364503992825384375679130606102141, 243939266812165980563239760114596239186104406092581261489726331066308569928747, 290372254706026096153481489592931453085780088683353495329679236536122868177482, 376644786442849392304182314585976636972636060283971456632342867625859504056500]",2,5,"115139476302519344463634772255798084337463636322414000002775671153459500424999","None","tpubDDPhf39HYJ34itcntagMVucwfUzvf6npJHGBMBuDaoUjyEeVFgvzgi8FbuQnRdMzMDd81kiqs7zcsGQRFC3B75QQQbVbtUM5cqnr7gWuqdf","p2wsh",1,1,'None',-1) ;
self: SHDSafeWallet xpub:tpubDDPhf39HYJ34itcntagMVucwfUzvf6npJHGBMBuDaoUjyEeVFgvzgi8FbuQnRdMzMDd81kiqs7zcsGQRFC3B75QQQbVbtUM5cqnr7gWuqdf, pubkeys: [b"\x0

In [112]:
invite  = {'public_key_list': [b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}", b'\x02\x1bP\xbbp\x89\xe5\xa2Q\x12\x1cp\x19\x03\xcd+\xc6Oa\xc7\x1a#\xaf\xcf$G\xa4\xed\x8bY\xa1\x14+', b'\x02\x81\xf8\xdf;r\x17\x1f\xf1\xe2_\xef\xe2\x98\xba\xa2X\xac\xf6\xe7[/\xca\xads\x1b\x8d\x12D\xc8\xf7\x1aJ', b'\x03@\xb5e\x00\xfbN\xe7\x02\xa5\xa3\xa1K\x1c\xf4\x13$;r)\xb7\xe8\xa3\x02~\xf2|Q\xf6\x0cF\xa8\xb4'], 'm': 2, 'n': 5, 'addr_type': 'p2wsh', 'master_pubkey': "tpubDDPhf39HYJ34itcntagMVucwfUzvf6npJHGBMBuDaoUjyEeVFgvzgi8FbuQnRdMzMDd81kiqs7zcsGQRFC3B75QQQbVbtUM5cqnr7gWuqdf", 'testnet': 1, 'segwit': 1}

In [113]:
invite

{'public_key_list': [b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}",
  b'\x02\x1bP\xbbp\x89\xe5\xa2Q\x12\x1cp\x19\x03\xcd+\xc6Oa\xc7\x1a#\xaf\xcf$G\xa4\xed\x8bY\xa1\x14+',
  b'\x02\x81\xf8\xdf;r\x17\x1f\xf1\xe2_\xef\xe2\x98\xba\xa2X\xac\xf6\xe7[/\xca\xads\x1b\x8d\x12D\xc8\xf7\x1aJ',
  b'\x03@\xb5e\x00\xfbN\xe7\x02\xa5\xa3\xa1K\x1c\xf4\x13$;r)\xb7\xe8\xa3\x02~\xf2|Q\xf6\x0cF\xa8\xb4'],
 'm': 2,
 'n': 5,
 'addr_type': 'p2wsh',
 'master_pubkey': 'tpubDDPhf39HYJ34itcntagMVucwfUzvf6npJHGBMBuDaoUjyEeVFgvzgi8FbuQnRdMzMDd81kiqs7zcsGQRFC3B75QQQbVbtUM5cqnr7gWuqdf',
 'testnet': 1,
 'segwit': 1}

In [159]:
consigners_reply = {'a': None, 'b': None, 'c': None, 'd': None, 'e': None, 'f': None}

consigners_reply["a"]=False
consigners_reply["b"]=False
consigners_reply["c"]=False
consigners_reply["d"]=False
consigners_reply["e"]=False
consigners_reply

{'a': False, 'b': False, 'c': False, 'd': False, 'e': False, 'f': None}

In [160]:
declined = False
declines = 0
m=2
n=6
for cosigner in consigners_reply:
    reply = consigners_reply[cosigner]
    print(reply)
    if not reply and isinstance(reply,bool):
        declines+=1
        if declines > n - m:
            declined =  True
            break
            
print(f"declined: {declined} because the amount of declines are {declines} and there are {n} participants")

False
False
False
False
False
declined: True because the amount of declines are 5 and there are 6 participants


In [158]:
bool(0)

False

In [1]:
from corporate_wallets import CorporateSuperWallet, ManagerWallet, SHDSafeWallet

In [2]:
words = "client sudden sunset borrow pupil rely sand girl prefer movie bachelor guilt giraffe glove much strong dizzy switch ill silent goddess crumble goat power"
manager2 = ManagerWallet.recover_from_words(words,entropy=256,passphrase="RobertPaulson",testnet=True)

6018716354834777059445516500463325153454936853151215624488541646839939367247233986759821493097378851875115552924255831805062990606698886137563075847103799


In [3]:
invite = {'public_key_list': [b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}", b'\x02\x1bP\xbbp\x89\xe5\xa2Q\x12\x1cp\x19\x03\xcd+\xc6Oa\xc7\x1a#\xaf\xcf$G\xa4\xed\x8bY\xa1\x14+'], 'm': 2, 'n': 3, 'addr_type': 'p2wsh', 'master_pubkey': "tpubDDPhf39HYJ34itcntagMVucwfUzvf6npJHGBMBuDaoUjyEeVFgvzgi8FbuQnRdMzMDd81kiqs7zcsGQRFC3B75QQQbVbtUM5cqnr7gWuqdf", 'testnet': 1, 'segwit': 1, 'level1pubkeys': [b'\x02\x1bP\xbbp\x89\xe5\xa2Q\x12\x1cp\x19\x03\xcd+\xc6Oa\xc7\x1a#\xaf\xcf$G\xa4\xed\x8bY\xa1\x14+']}


In [4]:
store_safev2 = manager2.accept_invite(invite, "Buenos Aires Manager2v3")

connection with database made.
INSERT OR IGNORE INTO SHDSafeWallet         (name,public_key_list,m,n,privkey,xpriv,xpub,addr_type,testnet,segwit,parent_name ,safe_index, level1pubkeys) 
VALUES("Buenos Aires Manager2v3","[242181816824935352779949905339585673702570459364503992825384375679130606102141, 243939266812165980563239760114596239186104406092581261489726331066308569928747]",2,3,"115139476302519344463634772255798084337463636322414000002775671153459500424999","None","tpubDDPhf39HYJ34itcntagMVucwfUzvf6npJHGBMBuDaoUjyEeVFgvzgi8FbuQnRdMzMDd81kiqs7zcsGQRFC3B75QQQbVbtUM5cqnr7gWuqdf",'p2wsh',1,1,'None',-1, '[243939266812165980563239760114596239186104406092581261489726331066308569928747]' ) ;
self: SHDSafeWallet xpub:tpubDDPhf39HYJ34itcntagMVucwfUzvf6npJHGBMBuDaoUjyEeVFgvzgi8FbuQnRdMzMDd81kiqs7zcsGQRFC3B75QQQbVbtUM5cqnr7gWuqdf, pubkeys: [b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}", b'\x02\x1bP\xbbp\x89\xe5\xa2Q\x12\x1cp\x19\x03\

In [7]:
nov_24_safe = store_safev2.get_daily_safe_wallet(201124)
res = nov_24_safe.open_partial_tx("c607c08a7aac67ae930416b93cdb7c62062758276f4e051f823a4892cdbac9a9")

prefix Nov24-of-20
connection with database made.
INSERT OR IGNORE INTO SHDSafeWallet         (name,public_key_list,m,n,privkey,xpriv,xpub,addr_type,testnet,segwit,parent_name ,safe_index, level1pubkeys) 
VALUES("Nov24-of-20_Buenos Aires Manager2v2","[242181816824935352779949905339585673702570459364503992825384375679130606102141, 243939266812165980563239760114596239186104406092581261489726331066308569928747]",2,3,"115139476302519344463634772255798084337463636322414000002775671153459500424999","None","tpubDF5r1amiE2pqGWuMuKZfj7EK3sP7hYqgBqr6xx6tpDLw4x5EYYMQpsGgeoJCErrSpQYAGipEBCtpFJW7KHBeFaRZoSA3QAotgvs1Mc3VEbS",'p2wsh',1,1,'Buenos Aires Manager2v2',201124, 'None' ) ;
self: SHDSafeWallet xpub:tpubDF5r1amiE2pqGWuMuKZfj7EK3sP7hYqgBqr6xx6tpDLw4x5EYYMQpsGgeoJCErrSpQYAGipEBCtpFJW7KHBeFaRZoSA3QAotgvs1Mc3VEbS, pubkeys: [b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}", b'\x02\x1bP\xbbp\x89\xe5\xa2Q\x12\x1cp\x19\x03\xcd+\xc6Oa\xc7\x1a#\xa

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/oscareduardosernarosero/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/oscareduardosernarosero/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/oscareduardosernarosero/Documents/US/Rice_U/blockchain/bitcoin/basics/final/wallet.py", line 114, in confirm_tx_sent
    self.db.delete_tx(tx)
  File "/Users/oscareduardosernarosero/Documents/US/Rice_U/blockchain/bitcoin/basics/final/wallet_database_sqlite3.py", line 329, in delete_tx
    raise Exception("Utxos are still saved as spent even though they have attempted to be set as not spent.")
Exception: Utxos are still saved as spent even though they have attempted to be set as not spent.



# Transaction from Daily-Safe Wallet GUI!

695e082bf8bafe95fc2e722edb44b4fcf8c3ba4ebad6c2ff40ec5431bce8e42b

8084da65d7c8f53ec110e02e874caabc1c3823c673179ac8e5e3d1329be52541


In [6]:
"01000000000101cce517291ed060c568f5a097db5b51b9a674c204275418efb3ca8bf01132716c0000000000ffffffff026a040100000000002200205ae31f55a2d06e4810746f9f41f5f496ecee2965a085c8122ab57858e3afe0d2d97e000000000000220020b331dd40d33b36c6ea281f31ecfde87b4a627caec1101cc14add0f0de47047af0300483045022100dee7a8772c472de754eeae45be117c6c06f278ef2cbbcfd951eb7bd30c97662402204a7034ae3ad67947dde45c1b68520b75d623ee514072f7ae01188fc0d5df72350147522102176e0cfd845d27ce83291f431dda33738fcf2096a6f3f7c4abdeb37008ede67d21036c54304d4822810fa9e88d2ddfdb0f5f2f9e93fc3e3b60596db8bb293653661f52ae00000000"

'01000000000101cce517291ed060c568f5a097db5b51b9a674c204275418efb3ca8bf01132716c0000000000ffffffff026a040100000000002200205ae31f55a2d06e4810746f9f41f5f496ecee2965a085c8122ab57858e3afe0d2d97e000000000000220020b331dd40d33b36c6ea281f31ecfde87b4a627caec1101cc14add0f0de47047af0300483045022100dee7a8772c472de754eeae45be117c6c06f278ef2cbbcfd951eb7bd30c97662402204a7034ae3ad67947dde45c1b68520b75d623ee514072f7ae01188fc0d5df72350147522102176e0cfd845d27ce83291f431dda33738fcf2096a6f3f7c4abdeb37008ede67d21036c54304d4822810fa9e88d2ddfdb0f5f2f9e93fc3e3b60596db8bb293653661f52ae00000000'

In [6]:
week_47_safe = store_safev2.get_weekly_safe_wallet(2047)

In [7]:
res = week_47_safe.open_partial_tx("c607c08a7aac67ae930416b93cdb7c62062758276f4e051f823a4892cdbac9a9")

connection with database made.
tx tx: c607c08a7aac67ae930416b93cdb7c62062758276f4e051f823a4892cdbac9a9
version: 1
tx_ins:
57922e786a57119273e13d641837dab1339b8995a55bf380e6585dc098c44d50:0
tx_outs:
77777:OP_0 a9da521259fbab8ee1738d6fa2932b9bac77bd30667841729e72741c4c112546
921352:OP_0 b331dd40d33b36c6ea281f31ecfde87b4a627caec1101cc14add0f0de47047af
locktime: 0
tx_ins [57922e786a57119273e13d641837dab1339b8995a55bf380e6585dc098c44d50:0]
connection with database made.
SELECT Utxos.tx_id, Utxos.out_index, Utxos.amount, Addresses.path, Addresses.acc_index, Addresses.address,
 Addresses.type 
FROM Utxos INNER JOIN Addresses 
ON Utxos.address = Addresses.address
WHERE Utxos.tx_id = '57922e786a57119273e13d641837dab1339b8995a55bf380e6585dc098c44d50' AND Utxos.out_index = 0;
[('57922e786a57119273e13d641837dab1339b8995a55bf380e6585dc098c44d50', 0, 999990, 'm/0/', 0, 'tb1q8kh9np5g72jnz8fenqdy8y7ayprsnafd34k5x8kdsqzvf2zfz8qq7mp9n5', 5)]
ini shm account _privkey: <ecc.PrivateKey object at 0x7fa28663

# Successfull tx from week safe



In [5]:
response_hex ="01000000000101504dc498c05d58e680f35ba595899b33b1da3718643de1739211576a782e92570000000000ffffffff02d12f010000000000220020a9da521259fbab8ee1738d6fa2932b9bac77bd30667841729e72741c4c112546080f0e0000000000220020b331dd40d33b36c6ea281f31ecfde87b4a627caec1101cc14add0f0de47047af030047304402203b253081b3b22cf018326110d405b5ef2d9d1938cd72a788c68d3967ad89055e022046868043183cf0eb8ddff9ed9f6a27ad017147ead5d6c82c0cd483918e4ba7f50147522102176e0cfd845d27ce83291f431dda33738fcf2096a6f3f7c4abdeb37008ede67d21036c54304d4822810fa9e88d2ddfdb0f5f2f9e93fc3e3b60596db8bb293653661f52ae00000000"

In [7]:
wrong_safe = store_safev2.get_daily_safe_wallet(201124)

prefix Nov24-of-20
connection with database made.
INSERT OR IGNORE INTO SHDSafeWallet         (name,public_key_list,m,n,privkey,xpriv,xpub,addr_type,testnet,segwit,parent_name ,safe_index, level1pubkeys) 
VALUES("Nov24-of-20_Buenos Aires Manager2v3","[242181816824935352779949905339585673702570459364503992825384375679130606102141, 243939266812165980563239760114596239186104406092581261489726331066308569928747]",2,3,"115139476302519344463634772255798084337463636322414000002775671153459500424999","None","tpubDF5r1amiE2pqGWuMuKZfj7EK3sP7hYqgBqr6xx6tpDLw4x5EYYMQpsGgeoJCErrSpQYAGipEBCtpFJW7KHBeFaRZoSA3QAotgvs1Mc3VEbS",'p2wsh',1,1,'Buenos Aires Manager2v3',201124, 'None' ) ;
self: SHDSafeWallet xpub:tpubDF5r1amiE2pqGWuMuKZfj7EK3sP7hYqgBqr6xx6tpDLw4x5EYYMQpsGgeoJCErrSpQYAGipEBCtpFJW7KHBeFaRZoSA3QAotgvs1Mc3VEbS, pubkeys: [b"\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}", b'\x02\x1bP\xbbp\x89\xe5\xa2Q\x12\x1cp\x19\x03\xcd+\xc6Oa\xc7\x1a#\xa

In [8]:
res = wrong_safe.open_partial_tx("30346826e0b88ba78dd2bd37bde9dd084b1ccd37624973cf1a392c19b9973a01")

connection with database made.
tx tx: 30346826e0b88ba78dd2bd37bde9dd084b1ccd37624973cf1a392c19b9973a01
version: 1
tx_ins:
8084da65d7c8f53ec110e02e874caabc1c3823c673179ac8e5e3d1329be52541:1
tx_outs:
29999:OP_0 a9da521259fbab8ee1738d6fa2932b9bac77bd30667841729e72741c4c112546
12724:OP_0 6953113107554a0bf78aed3731f494649e6022893adad4458de7a30ec02c5ee7
locktime: 0
tx_ins [8084da65d7c8f53ec110e02e874caabc1c3823c673179ac8e5e3d1329be52541:1]
connection with database made.
SELECT Utxos.tx_id, Utxos.out_index, Utxos.amount, Addresses.path, Addresses.acc_index, Addresses.address,
 Addresses.type 
FROM Utxos INNER JOIN Addresses 
ON Utxos.address = Addresses.address
WHERE Utxos.tx_id = '8084da65d7c8f53ec110e02e874caabc1c3823c673179ac8e5e3d1329be52541' AND Utxos.out_index = 1;
[('8084da65d7c8f53ec110e02e874caabc1c3823c673179ac8e5e3d1329be52541', 1, 43584, 'm/1/', 0, 'tb1q974rxefr7pddf4x3r0mgknwmvt2rdf0u7a8a5w02mqyuar7ep2qsssmnvr', 5)]
ini shm account _privkey: <ecc.PrivateKey object at 0x7fed5a797c

In [12]:
full_name = "Nov-24-of-20_Buenos Aires"
date_str,name = full_name.split("_")
month,day,of,year = date_str.split("-")

In [16]:
name

'Buenos Aires'

In [3]:
import datetime
from datetime import date
import calendar

In [20]:
month_n = list(calendar.month_abbr).index("Nov")

In [21]:
month_n

11

In [22]:
wallet_date = datetime.date(int("20"+year),list(calendar.month_abbr).index(month),int(day))

In [23]:
wallet_date

datetime.date(2020, 11, 24)

In [25]:
delta = datetime.timedelta(days=14)

In [28]:
today = datetime.date.today()

In [29]:
two_weeks_ago = today - delta

In [32]:
wallet_date > two_weeks_ago and wallet_date < today

True

In [4]:
week_date = date.fromisocalendar(2020,1,1)

AttributeError: type object 'datetime.date' has no attribute 'fromisocalendar'